# Finding Mutations
# B. Finding Mutations in DNA and Proteins

Code: Nhan TV.<br>
Email: TaVanNhan_sdh@hus.edu.vn.<br>

Compeau, Phillip. <i>Bioinformatics Algorithms: An Active Learning Approach by Phillip Compeau, Pavel Pevzner (2014) Paperback.</i> La Jolla, CA: Active Learning Publishers, 2014.

## Workflow

<a><img src="https://imgur.com/XwZtyQW.jpg" width="800" align="center">

## Table of Contents
1 [Multiple Pattern Matching](#matching)<br>
   &emsp;1.1 [Trie Construction](#trie-construction)<br>
   &emsp;1.2 [Applying the Trie to Multiple Pattern Matching](#multiple-pattern-matching)<br>
   &emsp;1.3 [Suffix Tries](#suffix-tries)<br>
   &emsp;1.4 [Suffix Trees](#suffix-tree)<br>
   &emsp;1.5 [Suffix Arrays](#suffix-arrays)<br>
2 [Burrows-Wheeler Transform (BWT)](#bwt)<br>
   &emsp;2.1 [Genome Compression](#genome-compression)<br>
   &emsp;2.2 [Constructing the Burrows-Wheeler transform](#constructing-bwt)<br>
   &emsp;2.3 [Inverting the Burrows-Wheeler Transform](#inverting)<br>
   &emsp;2.4 [Pattern Matching with the Burrows-Wheeler Transform](#pattern-matching)<br>
   &emsp;2.5 [Speeding Up Burrows-Wheeler Pattern Matching](#speed-up)<br>
   &emsp;2.6 [Locating Matched Patterns with BWT](#locate-bwt)<br>
   &emsp;2.7 [ Approximate pattern matching with the Burrows-Wheeler transform](#approximate)<br>

### 1 Multiple Pattern Matching<a name = "matching"></a>

<b>Nguyên nhân nào gây ra hội chứng Ohdo?</b><br><br>
Gần 1% các đứa trẻ sinh ra bị thiểu năng trí tuệ, nhưng vấn đề này vẫn gây khó khăn cho các nhà sinh học bởi chúng được gây ra bởi các rối loạn về gen khác nhau. Một trong những rối loạn này là hội chứng gây ra sự diễn đạt kém và khuôn mặt giống như một chiếc mặt nạ được gọi là hội chứng Ohdo. Vào năm 2011, các nhà sinh học đã giải được câu đố này bằng cách khám phá ra một số lượng các đột biến giống nhau của nhiều bệnh nhân. Các nhà nghiên cứu đã xác định được các đột biến của các đoạn cắt protein đơn cho hội chứng Odho [9].

Khám phá về nguồn gốc của hội chứng Ohdo này là một trong nhiều khám phá được thực hiện nhờ sử dụng <b>read mapping</b> trong nghiên cứu các rối loạn về gen. Trong read mapping, các nhà nghiên cứu so sánh các DNA reads được giải trình tự từ các cá nhân với <b>bộ gen tham chiếu (reference human genome)</b> để tìm ra các reads khớp (match) hoàn toàn với bộ gen tham chiếu này và các reads có thể chứa các đột biến từ một nucleotide này sang một nucleotide khác (single nucleotide polymorphisms, hoặc SNPs). Ngoài khoảng 3 triệu SNP (0,1% bộ gen của con người), con người còn khác nhau bởi sự sắp xếp lại bộ gen (rearrangements), sự chèn thêm (insertions) và xóa (deletions) có thể mở rộng ra tới hàng ngàn nucleotide [10][11]. Tuy nhiên, toàn bộ phần này chỉ tập chung vào những thuật toán tìm kiếm các SNPs.

<b> Giới thiệu về Multiple Pattern Matching</b><br><br>
Như chúng ta đã biết, các cặp đọc có thể được cung cấp bởi máy giải trình tự gen. Các lần đọc sẽ tạo thành một tập hợp các chuỗi patterns mà chúng ta muốn khớp với bộ gen text. Đối với mỗi chuỗi trong patterns, trước tiên chúng ta sẽ tìm thấy tất cả các kết quả khớp chính xác của nó dưới dạng một chuỗi con của text (hoặc kết luận rằng nó không xuất hiện trong text). Khi đi tìm nguyên nhân của một rối loạn di truyền, chúng ta có thể loại bỏ ngay vùng xem xét khỏi các khu vực của bộ gen tham chiếu nơi match chính xác xảy ra. Sau đó, chúng ta sẽ khái quát bài toán này để tìm các kết quả gần đúng, trong đó có các nucleotide đơn bị thay thế trong các lần đọc so với bộ gen tham chiếu (hoặc đại diện cho lỗi đọc).

Cách tiếp cận ngây thơ nhất cho bài toán Multiple Pattern Matching là thực hiện vòng lặp với thuật toán Pattern Matching, thuật toán này được gọi là Brute Force Pattern Matching. Ở đây, chúng ta sẽ trượt mỗi pattern dọc theo text, kiểm tra xem liệu có chuỗi con nào bắt đầu từ mỗi vị trí của text khớp với pattern hay không. Thời gian chạy của thuật toán này là $\mathcal{O}(|Text|.|Pattern|)$, vì vậy runtime của Brute Force Pattern Matching sẽ là $\mathcal{O}(|Text|.|Patterns|)$ trong đó |Text| là chiều dài của Text, còn |Patterns| là tổng chiều dài của tất cả các Pattern. Vấn đề với việc áp dụng thuật toán Brute Force Pattern Matching phát sinh khi |Text| và |Patterns| rất lớn. Trong trường hợp gen người (3GB), tổng chiều dài của toàn bộ các reads có thể vượt quá 1TB, dẫn đến bất kỳ thuật toán nào với thời gian chạy $\mathcal{O}(|Text|.|Patterns|)$ đều trở nên quá chậm.

#### 1.1 Trie Construction<a name = "trie-construction"></a>

Thời gian chạy của Brute Force Pattern Matching rất lâu là vì mỗi chuỗi trong Pattern phải đi qua tất cả Text một cách độc lập. Nếu bạn nghĩ về Text như một con đường dài, thì Brute Force Pattern Matching tương tự như tải từng pattern vào chiếc xe riêng của mình khi lái xuống Text, một chiến lược không hiệu quả. Thay vào đó, mục tiêu của chúng ta là đưa các mô hình lên xe buýt để chúng ta chỉ cần thực hiện một chuyến đi từ điểm bắt đầu đến cuối Text. Nói cách khác, chúng ta muốn tổ chức các Pattern vào cấu trúc dữ liệu để ngăn chặn nhiều đường truyền xuống Text và để giảm thời gian chạy.
Cuối cùng, chúng ta sẽ hợp nhất các Patterns thành một đồ thị không chu trình có hướng được gọi là Trie (phát âm là "Try"), được viết là Trie(Patterns) có các thuộc tính sau:<br><br>
&emsp;+) Trie có một nút gốc duy nhất với indegree 0, ký hiệu là root.<br>
&emsp;+) Mỗi cạnh của Trie (Patterns) được dán nhãn bằng một chữ cái của bảng chữ cái.<br>
&emsp;+) Mỗi chuỗi trong Pattern được đánh vần bằng cách ghép các chữ cái dọc theo một số đường dẫn
từ gốc trở xuống.<br>
&emsp;+) Mọi đường dẫn từ gốc đến lá hoặc nút có outdegree 0, đánh vần một chuỗi từ Patterns.

<a><img src="https://imgur.com/SPpldvA.jpg" width="400" align="center"> 

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Find all out-going edge from a node.
def get_out_edge(node, graph):
    all_out_edge = []
    for edge in graph:
        if node == edge[0]:
            all_out_edge.append(edge)
    return(all_out_edge)

In [3]:
# Get end node of out-going edge and corresponding label.
def get_end_node_label(current_node, all_out_edge):
    if all_out_edge == []:
        end_node_label = [[current_node, 'None']]
    else:
        end_node_label = []
        
        for edge in all_out_edge:
            """If there is a out-going edge from a node, 
            we will append end node of that edge and corresponding label"""
            if current_node == edge[0]:
                end_node_label.append([edge[1], edge[2]])
            else:
                end_node_label = [[current_node, 'None']]
                
    return(end_node_label)    

In [4]:
# Finding out-going node that its correspoding label is the same as current symbol.
def get_out_going_node(end_node_label, current_node, current_symbol):
    logic = []
    current_node = []
    # Finding out-going node that is end node of out-going edge 
    for end_node in end_node_label:        
        if current_node != end_node[0] and current_symbol == end_node[1]:
            current_node.append(end_node[0])
            logic.append("True")
        else:
            current_node.append(current_node)
            logic.append("False")
    """Logic = True if out-going node that its corresponding label is the same as current symbol,
    else with logic = False"""        
    if "True" in logic:
        lg = True
        node = current_node[logic.index("True")]
    else:
        lg = False
        node = 0
        
    return (lg, node)

In [896]:
# Constructing a trie from Patterns.
def construct_trie(patterns):
    
    trie = []
    leaves = []
    new_node = 0
    
    for pattern in patterns:
        current_node = 0
        
        for i in range(len(pattern)):
            current_symbol = pattern[i] # Get current symbol.
            all_out_edge = get_out_edge(current_node, trie) # Get all out-going edges.
            # Get end nodes of out-going edges and corresponing labels
            end_node_label = get_end_node_label(current_node, all_out_edge) 
            # Finding end node that its correspoding label and current symbol are the same.
            lg, node = get_out_going_node(end_node_label, current_node, current_symbol)
            # If logic =  True, only setting current node to end of out-going edge.
            if lg == True: 
                current_node = node
            # If logic = False, we add new edge to the trie and set current node to new node.
            else:
                new_node += 1
                trie.append([current_node, new_node, current_symbol])
                current_node = new_node
        # Finfing leaves of the trie.
        leaves.append(current_node)
    return(trie, leaves)

In [897]:
# Priting graph.
def print_graph(graph):
    for edge in graph:
        string = (str(edge[0]), str(edge[1]))
        print("->".join(string) + ":"+ edge[2])

In [898]:
data = pd.read_csv("D:/Data Science/Data/Mutations/How Do We Locate Disease/dataset_294_4 (2).txt", sep=" ", header=None)
patterns = data.iloc[:, 0].tolist()

In [899]:
print(data.head())
print(data.shape)

                                                   0
0  CGCGGCTTTTCAGGATTCAACACTTGTTTAGTTTCGGGTCTTAACA...
1  GTAGTTTTATCAAGGCGTCTTAACCGATTCATTGGGCTAAACGCTT...
2  ATCCAAACACCGACAATCAAGAGCCTCCGTAGCTCCCAGTGGCGAG...
3  GCCGAGTGTGGATCGGAAATGAGTTTTCCTGTGCTCTCCTTTTTCC...
4  CACAAGGCGTTCGTCTGTATACTGTTTGCTAGGCACAATGGTGTTC...
(82, 1)


In [900]:
trie, leaves = construct_trie(patterns)

In [901]:
print_graph(trie[183:193])

183->184:A
184->185:G
185->186:G
0->187:A
187->188:T
188->189:C
189->190:C
190->191:A
191->192:A
192->193:A


### 1.2 Applying the Trie to Multiple Pattern Matching<a name = "multiple-pattern-matching"></a>

Đưa ra một chuỗi Text và Trie (Patterns), chúng ta có thể nhanh chóng kiểm tra xem có bất kỳ chuỗi nào từ Patterns khớp với một đoạn bắt đầu từ tiền tố của Text hay không. Để làm như vậy, chúng ta bắt đầu đọc các ký tự từ đầu Text và xem chuỗi ký tự này đánh vần từ gì dọc theo đường dẫn đi xuống từ gốc của Trie. Với mỗi ký tự mới trong Text, nếu chúng ta bắt gặp kí tự này dọc theo một cạnh dẫn xuống từ nút hiện tại thì chúng ta tiếp tục dọc theo cạnh này; mặt khác, chúng ta dừng lại và kết luận rằng không có chuỗi nào trong Patterns khớp đoạn Text từ điểm đầu tiên. Cuối cùng chúng ta sẽ tìm được các đoạn text bao gồm các kí tự chạy từ gốc đến các lá của Trie, đó cũng chính là các chuỗi trong Patterns khớp với Text từ tiền tố của nó, thuật toán này được gọi là PrefixTrieMatching.

<a><img src="https://imgur.com/LP2D9Vy.jpg" width="700" align="center"> 

Tiếp theo, ta sẽ áp dụng thuật toán trên cho các chuỗi con của Text lần lượt từ vị trí đầu tiên đến vị trí cuối cùng của Text, thuật toán này gọi là TrieMatching. Chúng ta cần |Patterns| bước để xây dựng Trie(Patterns), mà chứa nhiều nhất |Patterns| nút, trong đó |Pattens| là tổng chiều dài của các chuỗi trong Patterns. Mỗi vòng lặp của PrefixPatternMatching chiếm nhiều nhất |LongestPattern| bước, trong đó LongestPattern là chuỗi dài nhất trong Patterns. TrieMatching gọi |Text| lần tới PrefixTrieMatching, như vậy tổng số bước bằng |Patterns| + |Text|.|LogestPattern|. Thời gian chạy này là một sự tăng tốc đáng kể so với |Text|.|Patterns| bước, khi |Patterns| lớn hơn nhiều so với |LogestPattern|. Thuật toán Aho-Corasick được phát triển năm 1975 tiếp tục giảm hơn nữa số bước sau khi xây dựng một Trie từ $\mathcal{O}(|Text|.|LongestPattern|)$ bước tới $\mathcal{O}(|Text|)$ bước [12].

In [732]:
# Determining whether a symbol of text matches a edge's label of trie or not.
def get_trie_matches_symbol(v, symbol, trie):
    lg = []
    node = []
    for edge in trie:
        """If a symbol of text matches a edge's label of the trie,
        we append end node of the edge to list of node."""
        if edge[0] == v and symbol == edge[2]:
            lg.append("True")
            node.append(edge[1])    
        else:
            lg.append("False")
            node.append(0)
            
    """If any label of out-going edge matches the symbol, 
    We select end node of the edge.""" 
    
    if "True" in lg:
        logic = True
        end_node = node[lg.index("True")]
    else:
        logic = False
        end_node = 0
    return (end_node, logic)

In [734]:
# Finding patterns matching sub-string from the beginning of text.
def get_prefix_trie_matching(text, trie, leaves):
    # To set initial variables.
    i = 0
    symbol = text[i]
    v = 0
    
    while 1 < 2:
        i += 1
        w, lg = get_trie_matches_symbol(v, symbol, trie)
        if v in leaves:
            return(patterns[leaves.index(v)])
        elif lg == True: # If any label of edges matches symbol of text.
            if i < len(text):
                symbol = text[i]
            v = w
        else:
            return("No matches found")

In [735]:
# Finding the first position of a text that a pattern matches to sub-string of the text.
def get_trie_matching(text, patterns):
    # Constructing a trie from patterns.
    trie, leaves = construct_trie(patterns)
    index = []
    count = -1 # Count for index of text.
    while text != '':
        # To get results from matching: patterns and sub-strings of text.
        pattern = get_prefix_trie_matching(text, trie, leaves)
        text = text[1::] 
        count += 1
        # if a pattern matches sub-string of text, we append index of text.
        if pattern in patterns:
            index.append(count)
            
    return(index)

In [736]:
text = "CACTCCTTTGCCTCAATATTCGCGGCCGTGCCCGACCGGGACGATTGGAATTCGGGCAAGGGATCGAGACGCGGAAAAATACCCAGTGCCGCGGTGCGGGGGATTGCGAGCCTAAAAGCATGCGAACTCCGCCAGGAAGATTGCAAACCAGTTCCGTGTGATACTTCCTACTTCCTACGCATTTACTTGCGTTTCGGCTGTAAACTAGCGCCATGCCGTCGGGTACTATTAGACGCCGAACTACTCAGTACAATTAATGTCTGGCTGTGGAAAGCACTCCCTGCCACCATGACTCGATAATTAACGTATTGGTGCGAGATTAACGCGCTAGCGTGGGCAAGGGGCAAGGGTTTACAGAGTTGGCAGTATTTGCGCCTAACTTCAAAAACATTCGCCATCCCCAACGCTACGCCCAGGTCATAAACTAGCTAGTTGGCACTGCCATATACTCGCGGGTCGCGGGTCACACATATCGCGGCGAATGCCGCCATAAAGTTACAATATGTAGTCTTCGAGCGTTTGATCTTTGATCTTGAGAGCCTAATATGGAATGGGTTGGTCGGCCGCGGTACAGGTGATCAAACGAGAATAATATTGTCCACCTCACGAAACTTCATTTCATTCCAGCCCGTGTTCCCGCTGCCGCTTGAATGTCTAATTAGTGAATGGGCATATCTCATAACCTCTGCCTTTTGCTTATGTAATAGTTCAGCTCGCTTCATGTAAAGCAAAACTTTCCAGTTAATCATGAGTCCATCACACCGCCAAGATTTGAAAACTGGGGGCACTTATATGTACATAAGTGCCCATCTTGCCATATGTCGCCTTACTAGGATCCGCCGACACATTCTCCGGACTGGTGGAACTCGGGCCCAGGCTAGCTGATTATACCAAGTCTTGGGGTGCGTGCTCCCCTGTACGCTTTTCAGTCGGCCGTTTTCTCGCGGGTCATGGTCACTAACAGGACCCATAACCTCTATATCCGACTAGAGAGTTTAAAACTAGCAAGGGATTCTCTGTGTCTATCCTACGTAACTCCCTGACTTCTAGACACGGTACGATGCTGTTGCAAGAACTGTCACTACTCTACTTCCTACGCGGAACACGTATATAGCATCTGGTTGACGCGACTTTAGGAGAACAGACCCCATTCCTGTCTTATACGCCCATAACTAGGGATCCCTGAAAACGCAGTCCCCAGCAGGTCTTAGCAGGATATTTTTGCGAAAGAAATGGGTGTCGCGATTAGTACTCTGGGCTTGGTCTATACTGGCCATTATGCAACACTACACCATCGGAATGGTCCTCCCAGTCAGGGGGCTCCCGGCGACTCGCCCGTTTGCGATGCAGTCGGAGTCGATGGCAGTTCTTCGCAGGCCAGTTTACGAGAACAGACTTACAGTGGATCTTTAATCGTCAAGCGTTCGAGTGTCAGCAAAATATCATAGACTGACCTCCAGTCAGCGGCCTTAGGCCATATCCCGGGATGGTTTCGTTCGCTGGAATCCACTGTCTCCAGTTCCGTCCGAGGCCGCCACCGATAAGTCCGCAGTTCTCCACAGATTTGGTCCCATGGCGTAGTGTTGTGCCCCGGCGAGCTGTACTGGTTCGCGTCAAATGTTACAAACCAAACTTCAGATTAACGACGTCACTCTATTTAATAGGCGCTGTAGACCGTGTTTTAGCAAAAACAATTCATACAAAAGCGCGGAGCGCGCCATTCGAAAGCACTGCAGAATGAAAGGCTGACCTGATCCCTATGCCCGCCGTATACGAGTACGACATGTACAACACGTGCCCTTTTTCCGAATGACCGTAAGGGAATGCAACATGGCCGCTTAACCTGAGGAGGACACAGCCTATTAGATAGCGCGTGTATACCCGATAAGAATCCATGGCAGAATCGGATAGTAGAGGGATACTGCGCGTGAATCGCAGGTAGCCAAATCCGCCCGACGGAATGAAAAGAGGAAAGCGAAGGTCGTAAGCTTCGTTGCGGTCGCAGGCTCTACATAAAATGACGCCTACATTCCGTAGGAACGCTCTAAATTAAGCGAAGTTTTACTTTTGTGTTGGGCGTAGCAACTGTCATTAGCTTCCAATAGATGTCGGCTACCTCTTTAGAGGTAGAATCAGACGAGTTCTCGCTAAGTTGTAGATCGGATCCAGCGATGACACACCTTTTTGGCCGACTCGCCCACAATACGCTCCGGACTCCGGACTTATGTTACTTTCGAATCCTGCCTTATTGATATACAGGCTCGGTGATTCATACTGCGTACAGCGTCTATGCTAGTCGAAGAATCACTGCCACCAGGCCACAAATGCCACGCGCGACACTTGAATCAGCTGCGTGTCGAGCATTGATTAACAACTAAGGATGCGGCGGCATATGTATAATTGGTCTACGTACCCCCTTACAGACGAACCTCGCCCCAGCCCATTAACTTTACGTGGTACTTTTAAAAGAAGATTCCCTCATTGCAACTCGACCGCTTCGGCTGACTAACAGCAGATATATGAAAGTGCTGGGCACCAATGGAATTGCGTAGAAGCAAGCCACTTTTTGGCGACAATTACGTCCCACGGTAACGGGCGGACCGCATTCGCGCGGTCCAATTATATCTGACATGTTTATTCATAATCCTAACATCTCCTATTTCCATTTCAGGCGACAAGAGCGGGTCCTAGCTTGAAGGAAGCCGCAGTATGTAGTAGAGAAATGGTGAACTTTCAGTAACGAGTCAGTCACGAGAGGTCGTCCAGGGCTTGGGTGCGGTCTCAATTCACCTTAAGATGACTGGTCTATGTTCAGCTCAGTAGTCCGCTCCTCTTTGTTTGTTTAACCCCTGACCGGATTGGTAGTCACCCTGGTTCCTAATACGACCCAGACGGGCTGGCTTCGATCCTTGCGATCCGAAGTCCTATTTCGGGCCCTGGCGCTGAGCCAAGAGTCGCTCGTCGGCTTGAAGGCGCCAGCAGATTTCGGCCTTCTGGGTGAGGTGCGATAATTGCATCATTTAGAACATAGCCTACTTGATCTGCTCACCCCGGTACGGCCTAAGACCTGCTAAGCCCGCGTTCGCCTACAGATTGGCCGCTTCAGCATTACCAGGCGGGCAAGGGGCAAGGGAGTCTTCATAAAAATGCCAGCTAGGTGCACTTCCCGATAAGCGAGAACAGAGAACAGATACGCCGCTCGTCTTGCTCCTGCAGAACCCTGGAAACGGTGCTTGTATCAGCTCCCCTCGGACCTAGGTCGTGTATTGGCTAGGCTTGACGGGTATTTCGATCTATGTCCCTAACGGCCACCATTCTAGGACGGGGTAATAATACCCGTGAGAACCATTTTACTGCGTCTAGGGTGCGTCCACTCGCTCAATAGTGCAAGTTCTTGGTTGCACCTTTTCTCCGGACTTGCTCTTGTACAGATCCCTACCCACGTCAGGAGAACAGAATGGAAGTACTTAGAGAGATCTACTGTTGCCTGGTTACCTAAATCACAACATATCACTTGGTGATAATGGCCCCCGTTGTACCAGGGCGGTTACATAATAAGTCCATGTCTAAGTTAGCGTGCAGTACCCAGCAAATCGAGGAGCATTATCGGTACGTTCGATTATGTGATGCTTCTGATTGGGTGCCCCATCACGCGCAGCCCATACGACCAGAATGAACGCCCTTACGCTGTTATAATGTCTACTAGCCCGCCCTACATAATTATATCTATCGGAGTGCGCTCACCACTTAGTCCGATAGCACGCAGGATCGCCTACTCAGTAAGGATCAAGACATAACTCTACCCCTCGCGGGTCGCGGGTCCGTACGACTACCGACTGCTGGGGCAAGGGGCAAGGGTATCAACTGGGTGTGCCGGGATGGTTAAAAGTCCATCGGAGCCTAAGCTATACGCCATGTCCAGCCATTGAAGTTAGGCTTCTCTCCCGATAAAAAAGTTCGTCAAATGGTATTTCCACGGAATCACGGCCTAGATTCAGTTATGGATTGAGGATTTAAAGAGAGCGTAATGCAGATAGTCGCGGTGCGTTATGTCGCGAGGAGCGACACTTTGCCCGACGACCCTGCTAGTATTCCGCGCATATGAGGTAGTCTCCTAAATATCCTGTCTGGCTTGTGCTGGATGAGGCGAAAGAACTGTAACGCGCCCGTGGGGCATGTGATGTCCTCGGGTACTTAGATGGTCTTGGTCCTAGGGCGGCGTTCTCAGCCGCTCATTGGGTTTACAACTTTGATCTTTGATCTTCCCAGGTGCGAGCAACTCCATGCGGTGATGGAGTCGTGGGGTTGTTTCCTGTTGATGGTGGCCGACCGAGGAATGCCCAATAAGTCGATCTCGCTCGTATCCCGATCCCAGGATGGAGATTACCCTGTGCGAAGAGGTCGGTAGGTCAAAGCTCTTAACATGGTTTTTCCCTTATGCCTCGCAATTGAAAACCGGAACACTCATGACCGTGGTCGATCATATGAAATAAAACCCCCGGCAGAGTATGAATATGGGCAAGGGCCCTCAAGCCCCGAGGCGCAGACCGCGGGAGACTATCTGCACTAACCTTCGGTCCCACCTCTATTAGGGCGCGCTCATACTGCGTTTGGACTGAGACCGATAGAATACTACTTTGATCTTCAGTACGATCGCATCATGGTGCCTTACCAATGATCCACTAGTGCGCAAAATAACCGGTTATAACCAAACTGAACCCGATTACTTAAAAGGCCGAAAGTACTTCCTACTTCCTAGAGAAAACGAGTGTTCACGACGATATTGGTCCAATTGAGATGCTTTTTTTTCACAAGGGGATTTGCCTGGAGACACACGGATCTCGGATGGGGCGGACATAGCTTACTACACCTGTTAGACACTGATAAGTTAGGGTGGATAATGCACCGTAGGCAAGGAGTACGAGCTTAGAAGTCTGAGCGCGAGAACAGAGTTTGATCTTTGATCTTCAATCTTGACAGTATTAGGGCTCACTCCGATGGTTAGTTCGGTAAAGTGACAAGAGATGTTTCTATAGGTGCAGTTGCATCTCACACGGTAACAACGGCTGACACGGCGGAAATGCGGTGAACTAACACGACTGTCACTCTGGGGTTGCGGCTACACAACAGATGTAAAGTCAAGTCAGCCATGGGAGGATACCCGGGCTACCCTATCTCTCAGTGGGGCCAATCGGTCTTCGGTGCAAGTATGCATCTTAACCTCTGTAAGACGTAACCACAACCTCCTCATGGTACTACACAGTACATGCGAGTTAGTTTCGGTATTCGAGCGCGTGACTGTATATGACCATGATGCTGTGTGAGGTTTCCTGTCGAACGAGTGCTATCCCAATCTTTATAAGCGTAGGAGGTCCCTAAAGCGGGTGTTTGCATGAAGTCCTGTTGTGTTTCGAATAGAGTATTCATAGAAACGAAACGTTAACCCATCGTGTGAAAAGAGCGGTCGATGTTGACGAGAGTGGAAGATCGTGTCCCTGTGAGGGCTACTTCCTACTTCCTACGATTGCCTTTCTATCCTTACATGCTAACATCTAACATCTGCTGCTTCTCTGACCTCACGCGGCCAGCTTAAATGTTCCCTCAAATTCTGACTACACCAACGCACATCTTCTGACGGGCATGACAGGCGCAATTGGCGATCACGAAAGCGAAGGGTGTCCCGTGTGGCCGTTAGCGTCGCTCGCGGGTCGCGGGTCACCGGCTACGCACTTTAGCAGCGAAATAGAGTCACATCGTCGTTGGGAGTCATGGTATCCATAACGCCAGCTTGTTAGCACGACCTTTAGCTTACTCATAGTCTCGACCTAACTATTATTTTGATCTTTGATCTTGCACTTAGCTAACATCTGATTTCCGTGTGCCATAGGGGTAGTTACTTTCTCACCCACTGTGAAAAGTATTGCGATGAAGGCAACACTAGTTAACGGAATTCCTAATGAACATCATAGAACGAGAAATAAAGCAAAATCTGAACATTCTGAGGGCCGGATACCCTGACACCAAAATCTTTAAGCACGACCGGGCCAAATGTACCCCTTAATGCGATGTCTTCTATATTGATAAGGTATTGAACCTAATAGGCCCGAGAATCACATGCGTTATTACGCCACTCCTGATGATGCCCTTTTGGCCATAAAGTAGGGTAAACGACGAACACAAAGACGGGGCCCATGGAAATGATGTTTGTTCCCAGATCGTGCTTTAAGTATGCTCAAAGACCAATGTGGGCTATGTCAACGGGGGAGAATTACGCTAGCGTCCTATTATCGGCAACCCTCCTTGTATCCAGAGGTCCATACAAAGTCGTGCTTGCCTATCTCACTTTTTTACGGGAAGCCATCAATGTACCTGACTGGGGCGGCGTAAAACAGCGGGACGGACGTGCTCTTTCGTCCCTGTAGGCGCCTGCTATACTGGCAGGCTCACTGTTTCGCTGTCCCTATAGCCGTGGTTGTTACATTCCGATACGACCCGGGTGGCCGTCGTGCATAATCGGTCCATCCGCTAAGTAGGACGGAGTCGCGGATCTTCCATCATAAAAACCTCCTGCCTTAGGAAGCACAGGTTTAGAAGACAATCGTTTAAAGCTGCAGCTGGATAGGCAGGCAGAACTCCTGAAGTGGCACATCTTAAAGACACGAATAACACTCCGGACTCCGGACTGAACATCTTGTTACTTGCCCGCACAATTATAACTGACAACCTGACTCCGCCGCCGAATCCACATAGTGGGAGAGTACCTCGAGTTACCAACGGAAGGCATTTCATACTTCCTCCAAGGATAGCAAGGAAAGTCAGGGTCCCGACTAGGGGACCTTGGTTAGAATGTCCGGATGATTAATGCGCATCCTTCTACTTCCTAAGGGGTACGTAGGTTCGTTCACGGATGGAAATTTCGCGGGACGAAGTGACTTACGCCCGCTCTTCGAAACAGCCGACGTCTGGCTTAGGCCCGCCGGCATTGTTCGGAACAGAGACGAGTTCGATGAGCATGGTTCTTCATGGTCACGCAGGGTCAGGTATCAGACGTCCACCCATTACCTTGACCTTCTCATTACGCCGCTTGAATGGCCAAGCTCATCTTCGAATAAGGTGTGCACGCTAATGTCGTAACATTGCCTCTGAATAGCCTATCAGCAAGTTTCCGTCCGGCGCAGTCGGTTAGAGTGCGGAAATTTCAAACATGTAATCTGACTTACTAATTAGGGCACCCGATGATCTCTGTGGGCATCATATCTCTTCTGGTGCCGCTCGCATGTCAGTTACTCGGATACTCTAGTAACGACGCGTGTACGGGGGCGGAATGAATTGATGATGTTTTCTGCAACATTTAGCGCGACATAGCAATTGCTTGGAAGGTCATGTCCATCGGCCTATCGACCGACTGCCTTCATGACTCCTCTCGCGCCATCCGTTTGATGGCTGTGAATGGTTCCACTATCAGGTGATAGAGGACAACGTACCTCATTCACGCCTCCAAGTGGCCTGAGTATGCGCGGCTTAATCCCTACGACATGCCATGTTAGAACTTGTCGTACCACTGTCGTATCGCGAGCTCTTGTCCTGTGGCTATTTCACTGCGCAGAAACGGTCCAATCTACGGCAATGCCGTTATACACGAGGCACGAAGTCGAGTGCGTGTCTAATGAATAGGTGAGTTCAAGGATAGGCGTGGAGGTCGTAAAGCAGCCAATGGGGCCTACAACACAAAGAAGATCTATAAATCGTGTGAGAACAGAGGGTACGAACGACGAAAAAAGACCAAAGGGGTTATGTGGGAGATAAGTCATTCCAGAGCAGCGTCACTTACGCAAAGCAAAAAGCAAAAGGATACTTGGTGCATGCCGTTAGTCTAAGTCGACATCAGAAACCAGTCCCAAATACACGTCGATGAGCGTGGCGAGAGGCGGCATCTCACTTACAAGTAGATAACTAACATCTAACATCTCCCGGCTGTCACTCGCTACAACCTGGGAGCTGCTCAGAGCAATAAAATGTCTTGCTATAAAAAACCGTTTGGGGCCAGGTAAGGACATTAAGGAACGCCTAAAAGATGCGCGACGCACTGACAGCTTCTCAATGTAGTCACGCTTCGAGCTCGGGCTGTGCCCACTTATGATACACTTCGAAACTCACACTGCCGGGTATATATGCACTTGGATTAAGCGCGACTGGCACAGACTACTGCGCTCAGGCGGTTAGTGCAGCGCGGCCATGGGCGTTGCCTTGACGTATCGCTCCAATCGCTGAACAAGTAAGCGCATCGGTAATGGCGAGTTGGAGAGCCCACAGTGCAGGGGGGCTCGCGGGTCGCGGGTCGCGACCCCACCAATGGGCGTGCGTTCGGACACCACCGCATTATATTCACAAAGATGCTCAATGAATAATCTGGATCCTATCCTCCTTTGAGACCAGGGGCTGTGGCGGACCTGCAATTAGAGATACAAGGAAACGCACTCCCACCGTTCTGGGCTGAAAAAGAGTTGTGATTCTAGGATGGGAACTAATAACCCGGCTGAGGCGAT"
patterns = ['GGGCAAGGG', 'TACTTCCTA', 'AAAGCAAAA', 'CTCCGGACT', 'GAGAACAGA', 'TCGCGGGTC', 'TTTGATCTT', 'CTAACATCT']

In [737]:
index = get_trie_matching(text, patterns)
print(index)

[53, 161, 168, 334, 341, 449, 456, 518, 525, 724, 849, 941, 1087, 1136, 1386, 2207, 2214, 2646, 3117, 3124, 3175, 3182, 3409, 3448, 3806, 3813, 3842, 3849, 4248, 4255, 4516, 4636, 4742, 4749, 4942, 4952, 4959, 5505, 5512, 5545, 5552, 5701, 5708, 5836, 5843, 5860, 5979, 6668, 6675, 6874, 7751, 7833, 7840, 7953, 7960, 8324, 8331]


Mặc dù TrieMatching nhanh nhưng việc lưu trữ một Trie sẽ tiêu tốn rất nhiều bộ nhớ. BruteForcePatternMatching làm việc với single read tại một thời điểm với bộ nhớ thấp vì chỉ cần lưu trữ bộ gen trong bộ nhớ, trong khi TrieMatching cần lưu trữ toàn bộ Trie trong bộ nhớ mà tỉ lệ với |Patterns|. Vì một tập hợp các lần đọc cho bộ gen của con người có thể tiêu thụ trên 1 TB nên bộ nhớ cần thiết để lưu trữ Trie là rất lớn.

#### 1.3 Suffix Tries <a name = "suffix-tries"></a>

Vì việc lưu trữ Trie (Patterns) đòi hỏi rất nhiều bộ nhớ, chúng ta hãy thay thế Text thành cấu trúc dữ liệu. Mục tiêu là so sánh từng chuỗi trong Patterns với Text mà không cần duyệt Text từ đầu đến cuối. Nói một cách quen thuộc hơn, thay vì đóng gói các Pattern lên xe buýt và đi đường dài xuống Text, cấu trúc dữ liệu mới sẽ có thể dịch chuyển tức thời mỗi chuỗi trong Patterns tới vị trí của nó trong Text.

Một <b>Suffix Trie</b>, ký hiệu là SuffixTrie(Text), là một Trie được hình thành từ tất cả các hậu tố của Text. Từ giờ trở đi, chúng ta sẽ thêm ký hiệu đô la ("$") vào Text để đánh dấu điểm cuối của Text. Chúng ta cũng sẽ dán nhãn cho mỗi lá của Trie theo vị trí bắt đầu của hậu tố có đường đi qua Trie kết thúc tại lá này (sử dụng lập chỉ mục dựa trên 0). Theo cách này, khi chúng ta đến một lá, chúng ta sẽ biết ngay hậu tố này đến từ đâu trong Text.

<a><img src="https://imgur.com/ZY4UQHc.jpg" width="600" align="center"> 

In [71]:
def get_patterns_from_text(text):
    
    patterns = []
    for i in range(len(text)):
        patterns.append(text[i::])
        
    return(patterns)

In [72]:
# Constructing a suffix trie from a text.
def construct_suffix_trie(text):

    trie = []
    leaves = []
    new_node = 0
    patterns, prefix = get_patterns_from_text(text)
    
    for pattern in patterns:
        current_node = 0
        concat_symbol = ''

        for i in range(len(pattern)):
            current_symbol = pattern[i] # Get current symbol.
            all_out_edge = get_out_edge(current_node, trie) # Get all out-going edges.
            # Get end nodes of out-going edges and corresponing labels
            end_node_label = get_end_node_label(current_node, all_out_edge) 
            # Finding end node that its correspoding label and current symbol are the same.
            lg, node = get_out_going_node(end_node_label, current_node, current_symbol)
            # If logic =  True, only setting current node to end of out-going edge.
            if lg == True: 
                current_node = node
            # If logic = False, we add new edge to the trie and set current node to new node.
            elif lg == False:
                new_node += 1
                trie.append([current_node, new_node, current_symbol])                 
                current_node = new_node
        # Finfing leaves of the trie.
        leaves.append(current_node)

    return(trie, prefix)

Lưu ý rằng nếu một chuỗi đơn Pattern khớp với một chuỗi con của Text bắt đầu ở vị trí i, thì Pattern cũng phải xuất hiện ở đầu hậu tố của Text bắt đầu từ vị trí i. Do đó, chúng ta có thể xác định xem Pattern có xuất hiện trong SuffixTrie (Text) hay không bằng cách đánh vần từ gốc (root) đi xuống, nếu điểm cuối của Pattern vẫn nằm trên đường dẫn xuống một lá nào đó ta sẽ nhận được một Pattern khớp với Text bắt đầu từ hậu tố của nó. Quá trình được lặp lại cho tất cả các Pattern nằm trong Patterns.

Ví dụ: Tất cả các đường dẫn bắt đầu bằng "ana" cho thấy ba lần xuất hiện của "ana" trong "panamabanana $". Mở rộng các đường dẫn này đến các lá (hiển thị bằng màu xanh lá cây) cho thấy rằng vị trí bắt đầu của những lần xuất hiện này là 1, 7 và 9.

<a><img src="https://imgur.com/VKu2lz4.jpg" width="600" align="center"> 

Hãy nhớ lại rằng việc xây dựng Trie (Patterns) cần có thời gian chạy và bộ nhớ $\mathcal{O}(|Patterns|)$.
Theo đó, thời gian chạy và bộ nhớ cần thiết để xây dựng SuffixTrie (Text) bằng với tổng độ dài của tất cả các hậu tố trong Text. Có |Text| hậu tố của văn bản có độ dài từ 1 đến |Text| và có tổng chiều dài |Text| · (|Text| + 1)/2, như vậy runtime sẽ là $\mathcal{O}(|Text|^2)$. Vì vậy, chúng ta cần giảm cả thời gian xây dựng và yêu cầu bộ nhớ của SuffixTrie để thuật toán trở nên khả thi.

#### 1.4 Suffix Trees <a name = 'suffix-tree'></a>

Chúng ta có thể giảm số cạnh trong suffix trie bằng cách kết hợp các cạnh trên bất kỳ đường dẫn không phân nhánh nào thành một cạnh duy nhất. Sau đó, dán nhãn cạnh này bằng hợp nhất của các kí tự trên các cạnh. Kết quả của cấu trúc dữ liệu được gọi là cây hậu tố, được viết SuffixTree (Text) [13].

Để khớp một Pattern với Text, chúng ta sẽ xâu Pattern vào SuffixTree (Text) theo quá trình giống như được sử dụng cho suffix trie. Tương tự như suffix trie, chúng ta có thể sử dụng nhãn lá để tìm vị trí bắt đầu của các Pattern khớp với Text.

<a><img src="https://imgur.com/wBv2OQp.jpg" width="600" align="center"> 

So với suffix trie, số lượng nút có thể lên tới bậc hai của chiều dài Text, số lượng nút trong SuffixTree (Text) nhiều nhất là 2·|Text|. Do đó, bộ nhớ cần thiết cho SuffixTree (Text) là $\mathcal{O}(|Text|)$.

Suffix tree tiết kiệm bộ nhớ vì chúng không cần lưu trữ các nhãn của các cạnh được nối từ mỗi đường không phân nhánh. Ví dụ, suffix tree không cần 10 byte để lưu trữ các cạnh lần lượt có nhãn "mabananas"; thay vào đó, nó chỉ lưu trữ một con trỏ đến vị trí 4 của "panamabananas", cũng như độ dài của "mabananas". Hơn nữa, suffix tree có thể được xây dựng trong thời gian tuyến tính mà không cần phải xây dựng trước suffix trie.

Mặc dù suffix tree giảm yêu cầu bộ nhớ từ $\mathcal{O}(|Text|^2)$ xuống $\mathcal{O}(|Text|)$, nhưng về trung bình nó vẫn cần bộ nhớ gấp 20 lần so với Text. Trong trường hợp của một Bộ gen người 3 GB, bộ nhớ RAM sẽ là 60 GB, đây là một cải tiến rất lớn so với 1 TB mà chúng ta cần làm việc với Trie (Patterns), nhưng nó vẫn đưa ra một thách thức về bộ nhớ. Điều này cho thấy một bí mật đen tối của ký hiệu big-O, đó là nó bỏ qua các hệ số không đổi. Đối với các chuỗi dài như bộ gen người, chúng ta sẽ cần chú ý đến hệ số không đổi này, vì biểu thức $\mathcal{O}(|Text|)$ có thể áp dụng cho cả thuật toán với
2·|Text| bộ nhớ và thuật toán với 1000·|Text| bộ nhớ.

Phần code thuật toán dưới đây chỉ mô phỏng lại một suffix tree chứ chưa áp dụng thuật toán cải thiện đáng kể khối lượng lưu trữ như đã nêu ở trên vì nó khá phức tạp, chúng ta sẽ trình bày code này ở một phần khác.

In [73]:
# Constructing a suffix tree from a text.
def construct_suffix_tree(text):
    tree = []
    # We still need to construct a suffix trie.
    trie, prefix = construct_suffix_trie(text)
    # Transform a suffix trie to a suffix tree.
    for edge in trie:
        all_out_edge = get_out_edge(edge[1], trie)
        # Concatenate all edges of non branching path to only edge.
        if len(all_out_edge) != 1:
            index = trie.index(edge)
            concat_symbol = ''
            for i in range(index + 1):
                concat_symbol = ''.join((concat_symbol, trie[i][2]))
            tree.append(concat_symbol)
            trie = trie[(index + 1)::]
                
    return(tree)

In [74]:
text = "ATCTACCAGCAGTGAACATGGGAGGACCAGTAAGGAAGGCTTACCCTCGATGTGTTACAGACTCGTTCGTAGGGTGTATAACGCCGCCGCTGG$"

In [75]:
suffix_tree = construct_suffix_tree(text)
suffix_tree

['A',
 'T',
 'CTACCAGCAGTGAACATGGGAGGACCAGTAAGGAAGGCTTACCCTCGATGTGTTACAGACTCGTTCGTAGGGTGTATAACGCCGCCGCTGG$',
 'T',
 'C',
 'TACCAGCAGTGAACATGGGAGGACCAGTAAGGAAGGCTTACCCTCGATGTGTTACAGACTCGTTCGTAGGGTGTATAACGCCGCCGCTGG$',
 'C',
 'T',
 'ACCAGCAGTGAACATGGGAGGACCAGTAAGGAAGGCTTACCCTCGATGTGTTACAGACTCGTTCGTAGGGTGTATAACGCCGCCGCTGG$',
 'A',
 'C',
 'C',
 'AGCAGTGAACATGGGAGGACCAGTAAGGAAGGCTTACCCTCGATGTGTTACAGACTCGTTCGTAGGGTGTATAACGCCGCCGCTGG$',
 'C',
 'C',
 'AG',
 'CAGTGAACATGGGAGGACCAGTAAGGAAGGCTTACCCTCGATGTGTTACAGACTCGTTCGTAGGGTGTATAACGCCGCCGCTGG$',
 'C',
 'AG',
 'CAGTGAACATGGGAGGACCAGTAAGGAAGGCTTACCCTCGATGTGTTACAGACTCGTTCGTAGGGTGTATAACGCCGCCGCTGG$',
 'A',
 'G',
 'CAGTGAACATGGGAGGACCAGTAAGGAAGGCTTACCCTCGATGTGTTACAGACTCGTTCGTAGGGTGTATAACGCCGCCGCTGG$',
 'G',
 'CAGTGAACATGGGAGGACCAGTAAGGAAGGCTTACCCTCGATGTGTTACAGACTCGTTCGTAGGGTGTATAACGCCGCCGCTGG$',
 'G',
 'C',
 'AGTGAACATGGGAGGACCAGTAAGGAAGGCTTACCCTCGATGTGTTACAGACTCGTTCGTAGGGTGTATAACGCCGCCGCTGG$',
 'T',
 'GAACATGGGAGGACCAGTAAGGAAGGCTTACCCTCGATGTGTTACAG

#### 1.4 Suffix Arrays<a name = 'suffix-arrays'></a>

Năm 1993, Udi Manber và Gene Myers đã giới thiệu suffix arrays như là một sự thay thế hiệu quả về bộ nhớ cho suffix trees [14]. Để xây dựng SuffixArray (Text), trước tiên chúng ta cần sắp xếp tất cả các hậu tố của Text theo thứ tự alphabet, giả sử rằng "$" đứng đầu trong bảng chữ cái. Suffix arrays là danh sách các vị trí bắt đầu của các hậu tố được sắp xếp này:

<a><img src="https://imgur.com/eXODwwe.jpg" width="400" align="center"> 

Bài toán xây dựng Suffix Array có thể dễ dàng được giải quyết sau khi sắp xếp tất cả các hậu tố của Text, nhưng kể cả các thuật toán nhanh nhất để sắp xếp một mảng gồm n phần tử cũng yêu cầu $\mathcal{O} (n · log n)$ sự so sánh, sắp xếp tất cả các hậu tố cần $\mathcal{O} (| Text | · log (| Text |))$ so sánh.

Tuy nhiên, tồn tại một thuật toán nhanh hơn để xây dựng các Suffix Arrays trong thời gian tuyến tính và chỉ cần khoảng 1/5 bộ nhớ so với Suffix Tree, có thể giảm yêu cầu bộ nhớ 60 GB  cho bộ gen người xuống tới 12 GB.

In [1]:
import numpy as np

In [2]:
def construct_suffix_arrays(text):
    patterns = []
    for i in range(len(text)):
        patterns.append(text[i::])
   
    suffix_array = np.argsort(patterns)
    patterns = sorted(patterns)
        
    return(suffix_array, patterns)

In [214]:
text = 'GCCAGCCACACGCTAACGACACCCCGTCCACGATGGTAGTTTAGGTCATTCCAGGGCCACTTGAACCTTAGTCAGCTTGACTAGGTAAGGTATCTCCTTGTGCCGGGAAAGGTATCGTGCATAGGCTTAAGAAGACAAGCCGTTAGTTATCCTGTATTGCTCCGCCCGTTCAAAGCATCGGCTCGGAGGCCCAGAGTGGAGCGCATGTCTCGGAGACTTATCTGGCCACATTGATTGAGGTAAGCGAATCATCGATGGGTAGACTAAATGTAATTTTCTCGTCTATCTGGTCAAATGCCGCCAAGCAAGGTTAAAATTAGAGGCGCGCCACGGCTACAGTCTCGACCCATCCTCAATTATATCTGGGTTATACCCGGGACCACGCCGTCCTAACACCGGGTCATAACCGGAAACCAACGACAGTCACTTATGATCTACTGCATCGAGTGTCTCTCCGCACCCTCATCATCAACGCACTTGATAGGTGGATAGCATATGCTACGCATCCGGTACAGGCGCGGCACCGTCCAGGACAGTTATCGGATCCTGTGGCCGTCTAAGACCATCTCTGCTCGAGAGAGAGACAACAAGTTTCCTAAGCCTAGGCGCGCCCGCCGTAAGAGACACGCCGTCCGTCACTCTACCGCCGCGAGGATTTGGAGTAATCTCGTAGTGAGCGATCATCAACTTGGAGGCGGATATCCTCACTCGTCATGACAGGCATTGGTGTTTCTATTGAGGGCTCACTTATGATCAGTCGGCCGCAGTCTCGTGGTTTATCCCTTGGTCAACGCCCTGGCCCCCTTTCCTAAGCGTGTAGTACTTTCATTTATCGTAAGAGTACTTACTGGGCTACAAGGGGGGTGCCAGTTAGTCCTTTGCCCCGGACCCATTATAGCTATTCTCATATCGGCTAATTAATAACCACCTCCCCTATC$'

In [215]:
suffix_array, _ = construct_suffix_arrays(text)
print(suffix_array.tolist())

[938, 312, 410, 171, 107, 292, 265, 313, 585, 391, 411, 922, 404, 63, 14, 415, 470, 789, 685, 128, 131, 558, 618, 836, 302, 172, 136, 597, 241, 810, 856, 108, 86, 306, 588, 919, 246, 663, 293, 266, 915, 314, 354, 271, 583, 134, 854, 586, 18, 392, 623, 7, 716, 511, 419, 335, 532, 227, 412, 923, 378, 561, 344, 887, 20, 371, 458, 642, 405, 394, 522, 926, 64, 15, 416, 29, 471, 500, 790, 625, 381, 9, 329, 262, 79, 706, 637, 436, 846, 842, 215, 745, 425, 58, 475, 686, 821, 129, 581, 132, 621, 559, 260, 213, 579, 619, 577, 575, 318, 837, 192, 303, 490, 173, 3, 137, 598, 242, 675, 199, 811, 896, 73, 529, 651, 718, 186, 320, 603, 513, 692, 122, 52, 738, 857, 237, 82, 109, 87, 42, 482, 307, 660, 839, 818, 421, 69, 872, 755, 337, 765, 671, 194, 445, 868, 144, 534, 37, 589, 920, 402, 369, 488, 894, 120, 480, 698, 906, 358, 493, 935, 467, 682, 752, 464, 679, 247, 778, 504, 348, 700, 148, 543, 441, 250, 538, 908, 176, 831, 113, 432, 91, 664, 564, 219, 360, 284, 713, 749, 429, 294, 495, 254, 32, 267,

Khi chúng ta đã xây dựng Suffix Array của một chuỗi Text, chúng ta có thể sử dụng nó để nhanh chóng xác định vị trí
mỗi lần xuất hiện của một chuỗi Pattern trong Text. Đầu tiên, hãy nhớ lại rằng khi khớp Pattern với Suffix Trie, tất cả các khớp của Pattern trong Text phải xuất hiện tại điểm đầu tiên của các hậu tố của Text. Thứ hai là sau khi sắp xếp các Suffix của Text, các Suffix bắt đầu với một Pattern được nhóm lại với nhau theo trình tự alphabet. Ví dụ trong hình dưới đây Pattern = "ana" xuất hiện ở đầu các Suffix "anamabananas", "anana" và "anas" của Text = "panamabananas"; các hậu tố này xảy ra trong ba hàng liên tiếp và tương ứng với các vị trí bắt đầu 1, 7 và 9 trong Text.

Câu hỏi là làm thế nào để tìm các vị trí bắt đầu này cho một chuỗi Pattern tùy ý mà không cần lưu trữ các Suffix được sắp xếp của Text. Thuật toán PatternMatchingSuffixArray xác định chỉ mục đầu tiên và cuối cùng của Suffix Array tương ứng với các hậu tố bắt đầu bằng Pattern(các chỉ số này được ký hiệu first và last tương ứng). PatternMatchingSuffixArray là một biến thể của một kỹ thuật tìm kiếm chung gọi là tìm kiếm nhị phân (binary search), tìm một điểm dữ liệu trong một bộ dữ liệu đã được sắp xếp bằng cách lặp lại quá trình chia đôi dữ liệu và xác định nửa nào của bộ dữ liệu chứa điểm dữ liệu. Tuy nhiên thuật toán sẽ không chỉ ra được số lần lặp lại của điểm dữ liệu trong bộ dữ liệu, mà nó sẽ dừng lại khi tìm thấy vị trí đầu tiên của điểm dữ liệu trong bộ dữ liệu đó.

In [3]:
class MultiplePatternMatchingSuffixArray(object):
    
    def __init__(self, text, patterns):
        
        self._text = text
        self._patterns = patterns
        self._suffix_array, self._text_patterns = self.construct_suffix_arrays()

    def get_positions_of_patterns(self):
        
        position = []
        for pattern in self._patterns:
            first, last = self.match_pattern_to_suffix_array(pattern)
            position.append([first, last])

        return(position)
        
    def match_pattern_to_suffix_array(self, pattern):
        
        min_index = 0
        max_index = len(self._text)
        
        while min_index < max_index:
            mid_index = int((min_index + max_index)/2)
            if pattern > self._text_patterns[mid_index]:
                min_index = mid_index + 1
            else:
                max_index = mid_index
        first = self._suffix_array[min_index]
        last = first + len(pattern) - 1
        
        return(first, last)
    
    def construct_suffix_arrays(self):
        text_patterns = []
        for i in range(len(self._text)):
            text_patterns.append(self._text[i::])

        suffix_array = np.argsort(text_patterns)
        text_patterns = sorted(text_patterns)

        return(suffix_array, text_patterns)

In [341]:
text = 'TTCTATCGGTTCGAAGTCGGATCTTGAGGGCGGTGGGGTCATTAGGGATACAGTCCATTATTACGCGCGGTTCGCACTTATGACAGTCGCTCCCCTAGCGTTAGCAGAGCACAGGCATCCGAGGTTTAAAGTTTGGGGTGGTGGAAATACTTAGCCAAGGGGGCGGAGTCCAACTGCGCGAGCTCAATCGACCGTTAGAGGCCTCCGGCCTCCGGACCGTTCACGAGATATGCATGCTCGGTTTTAGTACGCGCAGGTAAGTCCACAGTCAAGTTCGAGATGCCCGACAAACTCATCGAGGATTAGAACGGCCTCCCTTCCTGTTCGTTCTGTTAAGGTTCGGAACGCTCCTGGGCTCAAGATAGTGTTAGGCGGAATGTTCTAGGTTCTAGGTTGTATTACGCACTTTCAAAAAGAAATGGCCTCCGGCCTCCGGGTTCAGTGTTTACGCCGTTCCGACATTTTGGAATACACGAACCCATATTCTGCAAGGCAATCACTGCGCTCCGGGGATTTGCACGGTCATGAACTTTTCGGCCTCCGGCCTCCGGTTAATGTCCGAAGGTTCTGACTCGGATTTCTTCCCATGTGTGCTTTGATACACCCCGGAACGCGCTTCGAGACTACAGGCGACTGTCTATCTACAACAACACCCAGCCCTGTGAGCCTCAAGTCGAACATGATCTTTGTGCTTTGTGCTTTGCCCCGCTAACTCACCTTCACAGAAGGGACGTGAACCAATTAACCGCATGCTCCATGACAGGTATGGTGCCAACTGGACCATGGTTTCAAGGGTTTAACCGATATGGCAATGGCCGCTTTCGCTCTGATCTTTCCATTTAGTGACCAGAGAAGCAGGATACGATGGTTATATTAGGGCACCAATAGCGAGTTTAGGCACCATGACCCAGGGGTCGGTCTCGCCTATCGCCCCATTACTCCTAGTTAAATGGGGTGGCCTCCGGATACACGATTGGCGCCAGGCCCCAGGGTGTTATGTCATACTACAGTTCAATGGTCCTCACGAATCACGGCTTCCATGTCCAGCGGCCTCCGGCGCAATTAATGTGTAAGATGCTAACAAGGCGATCATAAGCATAGTGTAACTGGCCGCACCCGGAACTCTTCCCCATGACCCATGACCCCGTCATCGGTATCGTCCGCAGATGAAGACTGCGGAGAGATATGTTATGGCAGCTAATTTATAAGGTCGCTTGGCTGAACCAGGCCCCAGGCCCCCCCGTTCTAGGTGCACGTCATTGCAACGCCAATATTTCGAGTGGGGAGTTTCAATGCCGCCCTCTATATAGAGATTTCCGGCATACATCGGGTCATTATAACGCGGCGAAAACCCTCGTCCCAGGCCCCAGGCCCCGCGATTGAGTGGTGCGCAGGTGTTTCACATTATTAGGGCTAGTTGCCACATGGCCCCTGTCCACTGAACAGTGGGCAATCCGACAAATTTGATCTAGCCCGTCCCGAGAGCCAGAGGTAACTGCAGCCACTCTAGAGAAAGCATGGGACGGGAGAAGGTGACGGGAGACGGGAGACTGGCTTACCTGTATAAATCCGCCCTCCTCTAGACGCTTTGTTCAGTGTTAGGTTCGGCCTCCATTACCTCCCCTTTTAGGTGCATGACAGGGGAGCATCTTTCAACGTTCCATGACCCATGACCCGAACAGACGGTTTATGTGCTTTGTTGTATCTTCAACGCAAAAGACGTCTACGAGCTCCTTGAGGGAAGCGGACATCCGCTAATCCCGATCCCGCGAGGACAGAGCAATGCATGGCTTAGAAAACCGCCTACCTGAATTCATCATGCACACTCATGACCCAATACATACAGGTTACATTACCTATGATAGGCTTCAATGGTTTTATCCTAGAAACCCCAGGCCCCAGGCCCCTCGAACTGTTGGCCTTGGAACAACTTACGTTCTTATTTACTGTTCGTGGCATGATCGGCCACCTATAGGCACATTCCACCTTCGAAGAGACTGTCCCCAAAGCGCAGGCTACACTTCGCTAAATATTGCTCAGTGAGTGATGGTCTTCCAATCCTAAAGAGCTGAATATGTATGGCACTGATATAGAAGTCGAATCTATAAATACAATACCTTCATAATTGTGCTTTGTGCTTTGAAGCGGATACCGAAAAGCTCTGTGAAAAAACCACCATCACCTATTAGACGGGAGACGGGAGAAAGCATCGCGTCCCACTGCAAGGGGACGCCATGACCCTGATCAAGCGCAGCTCGAGCAGTACGTAAGTACAGTGACGGACGGGAGACGGGAGACCGAAATTCAGTATATCAGCAAGGCGGGTATCCATGAAAAAGTACGTGTCGTGGCCTGGTAGCTTAGTAGAGTCCTCAACGCTATCGGGTAGGACAATTTTAAATACTTAGAAATTGGGGTAGGGGACAACAACACGATTTTTTTCTGTGCTTTGTGCTTTGCGTTGAATCAAAAAAGCTTGCAAGACTACCTGGGCTCTCCCCACGGCGTCAGCTTGCAATCCCGTATCCAAAGCTTCGCCAAAGATGGGGCGTAGCCTGCTGTCACCGGCCGCATCTGAGACCAGTCTTCTTAAGTAGGGCGGCGCGCTACTTTTACAGCATTTCGACTTCAGTGTTGGCCGACTCCATCCAACCTGGTCACCGAACAGCGCGGGTGTTTACGCTTTCGCACCGCCTTGCAATTCGCCGGACTAAGCGTAATACGCCGGTTGAAAACGGTGCTGCAGCCCCGGAGGGTCCTAGACGTGCGACTACTATAGCTAGAGTTAATGTCATATCACCTGTTTCAGTGTTAACAGGCGATAAATAGTTACAGAAGGAAACATCAGGTAGAGGTTTC'
patterns = ['GTTCTAGGT', 'GACGGGAGA', 'CCATGACCC', 'CCAGGCCCC','TGTGCTTTG', 'TTCAGTGTT', 'GGCCTCCGG']

In [342]:
positions = MultiplePatternMatchingSuffixArray(text, patterns).get_positions_of_patterns()
print(positions)

[[1242, 1250], [2195, 2203], [900, 908], [1223, 1231], [2133, 2141], [2804, 2812], [206, 214]]


In [343]:
print("Testing result: ")
print(text[1242 : 1251], text[2195 : 2204], text[900 : 909], 
      text[1223 : 1232], text[2133 : 2142], text[2804 : 2813], text[206 : 215])

Testing result: 
GTTCTAGGT GACGGGAGA CCATGACCC CCAGGCCCC TGTGCTTTG TTCAGTGTT GGCCTCCGG


Bibliography: <br>
[9] “Whole-Exome-Sequencing Identifies Mutations in Histone Acetyltransferase Gene KAT6B in Individuals with the Say-Barber-Biesecker Variant of Ohdo Sy... - PubMed - NCBI.” Accessed May 13, 2020. https://www.ncbi.nlm.nih.gov/pubmed/22077973.<br>
[10] Tuzun, Eray, Andrew J. Sharp, Jeffrey A. Bailey, Rajinder Kaul, V. Anne Morrison, Lisa M. Pertz, Eric Haugen, et al. “Fine-Scale Structural Variation of the Human Genome.” <i>Nature Genetics</i> 37, no. 7 (July 2005): 727–32. https://doi.org/10.1038/ng1562.<br>
[11] Montgomery, Stephen B., David L. Goode, Erika Kvikstad, Cornelis A. Albers, Zhengdong D. Zhang, Xinmeng Jasmine Mu, Guruprasad Ananda, et al. “The Origin, Evolution, and Functional Impact of Short Insertion-Deletion Variants Identified in 179 Human Genomes.” <i>Genome Research</i> 23, no. 5 (May 2013): 749–61. https://doi.org/10.1101/gr.148718.112.<br>
[12] Aho, Alfred, and Margaret Corasick. “Efficient String Matching: An Aid to Bibliographic Search.” <i>Commun.</i> ACM 18 (June 1, 1975): 333–40. https://doi.org/10.1145/360825.360855.<br>
[13] “(PDF) Linear Pattern Matching Algorithm.” Accessed May 13, 2020. https://www.researchgate.net/publication/229067733_Linear_Pattern_Matching_Algorithm.<br>
[14] Manber, Udi, and Gene Myers. “Suffix Arrays: A New Method for On-Line String Searches.” <i>SIAM Journal on Computing</i> 22, no. 5 (October 1, 1993): 935–48. https://doi.org/10.1137/0222058.

### 2 Burrows-Wheeler Transform (BWT)<a name = 'bwt'></a>

#### 2.1 Genome Compression <a name = 'genome-compression'></a>

Suffix Array đã giảm đáng kể bộ nhớ cần thiết cho tìm kiếm Text và cho đến đầu thế kỷ này chúng vẫn thực sự tốt trong việc khớp Pattern. Chúng ta có thể tham vọng đến mức tìm kiếm một cấu trúc dữ liệu mã hóa Text với bộ nhớ chỉ xấp xỉ bằng độ dài của Text trong khi vẫn cho phép khớp Pattern nhanh chóng hay không?

Để trả lời câu hỏi này, chúng ta sẽ xem xét chủ đề dường như không liên quan đến nén văn bản. Trong một kỹ thuật nén đơn giản được gọi là mã hóa độ dài chạy (run-length encoding), chúng ta thay thế một chuỗi k xuất hiện liên tiếp của ký hiệu s chỉ bằng hai ký hiệu: k, theo sau là s. Ví dụ, mã hóa độ dài chạy sẽ nén chuỗi TTTTTGGGAAAACCCCCCA thành 5T3G4A6C1A.
Mã hóa độ dài chạy hoạt động tốt đối với các chuỗi có nhiều lần chạy dài, nhưng thực tế bộ gen không có nhiều lần chạy. Bộ gen chứa các kí tự lặp đi lặp lại không liên tiếp. Do đó, thật tuyệt nếu trước tiên chúng ta có thể chuyển sự lặp lại này thành các lần chạy và sau đó áp dụng mã hóa độ dài chạy cho chuỗi kết quả.

Một cách ngây thơ trong việc tạo ra các chuỗi chạy là sắp xếp lại các ký tự trong chuỗi theo thứ tự alphabet. Ví dụ: TACGTAACGATACGAT sẽ trở thành AAAAACCCGGGTTTT, mà sau đó chúng ta có thể nén thành 5A3C3G4T. Tuy nhiên phương pháp này có vẻ chưa hợp lý. Nếu một chuỗi DNA GCATCATGCAT và ACTGACTACTG - cũng như bất kỳ chuỗi nào có cùng số lượng nucleotide - được sắp xếp lại thành AAACCCGGTTT thì chúng ta không thể giải nén chuỗi đã được nén, tức là đảo ngược quá trình nén để tạo ra chuỗi gốc.

#### 2.2 Constructing the Burrows-Wheeler transform <a name = constructing-bwt></a>

Chúng ta hãy xem xét một phương pháp khác để chuyển đổi các lần lặp lại của chuỗi thành các lần chạy
được đề xuất bởi Michael Burrows và David Wheeler vào năm 1994 [15]. Đầu tiên, tạo ra tất cả các vòng quay theo chu trình có thể có của Text; một vòng quay theo chu trình được xác định bằng cách cắt bỏ một Suffix từ cuối Text và nối thêm Suffix này vào đầu Text. Tiếp theo - tương tự như Suffix Array - sắp xếp tất cả các vòng quay theo chu trình của Text theo thứ tự alphabet để tạo thành một ma trận cỡ |Text|x|Text| của các ký tự mà chúng ta gọi là ma trận Burrows-Wheeler, kí hiệu M (Text).

Lưu ý rằng cột đầu tiên của M (Text) chứa các ký hiệu của Text được sắp xếp theo alphabet, đây chỉ là sự sắp xếp lại ngây thơ của Text mà chúng ta đã mô tả. Đổi lại, cột thứ hai của M (Text) chứa các ký hiệu thứ hai của tất cả các vòng quay theo chu trình của Text và do đó nó cũng thể hiện sự sắp xếp lại (khác nhau) các ký tự từ Text.
Tương tự, ta có thể chỉ ra rằng bất kỳ cột nào của M (Text) cũng là sự sắp xếp lại các ký tự của Text. Chúng ta quan tâm đến cột cuối cùng của M (Text), được gọi là Biến đổi Burrows-Wheeler của Text, hoặc BWT (Text), được hiển thị màu đỏ trong hình dưới đây.

<a><img src="https://imgur.com/IR6pFPU.jpg" width="500" align="center"> 

Nếu chúng ta kiểm tra lại biến đổi Burrows-Wheeler ở hình trên, chúng ta sẽ nhận thấy ngay lập tức rằng nó đã tạo ra chiều dài chạy "aaaaa":

<a><img src="https://imgur.com/NARGytE.jpg" width="400" align="center"> 

Khi biến đổi Burrows-Wheeler được áp dụng cho bộ gen, nó biến đổi nhiều lần lặp lại của bộ gen thành các lần chạy. Sau đó chúng ta có thể áp dụng một phương pháp nén bổ sung như mã hóa độ dài chạy để giảm hơn nữa bộ nhớ.

In [1]:
import numpy as np
from collections import Counter

In [2]:
def get_bwt(text):
    text_list  = []
    bwt = []
    for i in reversed(range(len(text))):
        text_list.append(text[i::] + text[0:i])
    sorted_list = sorted(text_list)
    for pattern in sorted_list:
        bwt.append(pattern[-1])

    return(''.join(bwt))

In [251]:
text = 'CGGGGAGTTTAGATCCTATGGCGAGGCAACTTCGTCCCCGACTGACATTTGAGTAACGCCGTCATATCATAGCTTAGCCACGGACGCCTACTTGAAGAATGCGAAAGCGGCTGCACGAGCTGTCCCTCAGCCTCACTCCATTATGGCCGGAGCGGTCAATGCACCCCACGCCAGGCTCTAAACGACGCACTGTAAAAACTAGGTTGCCGCTTAAGAGAAGAGTGTGTTTCGTGCTTAGTGCCCCCTATTGGCGCTAGCAGACGGCGTTCGCCCAGGTTTTTAACCGACCCAGCACGGCCGACCCCTAGAGGGATTAAATGGGCAGTTTGATGATGGGTGGGTAATAGTCGATGTGGGTGACGCCCTCTACGCTTGTTCCCGCATCACGTGTGGCTTTCCGTAGCCTGCAGAAACATCCATGGCAGGCATAAAACGGCCGATAAGTAGTGAAGAACTAGCTATTCCTCGCTTGGTATTCTCTTATGACTTACGGCATGATCTCCTTGTCAGAGAGCGATCGGGGTGTTCCGATGTTTCGTGAACACCTTTTAGTCAAGGGGGTTTTTAACGGTACGGGTGGACCTGAAGCAAGTGTGGTCCGTCCACAAGTATCTCGCTGAAAAGGCCCATGCGCGGTGCATTCAGCTGTATCCTCTGCCCAAAGCTTCATTCCCGGCCCGAGATAAGATGACGATTGCTCTCTAGCTATGAGGCTATGTGCTGTAGTGCCTGGTGTGTGAAAAAACCTTAATGCATATCACTCCATGGTAGCTATAACGACTAGACAGTCACGCGCGGACTATATAATTTTTTGTAGATCTGTTATACGTGGGATTTACCCGGCGAGATAGTGAGGGTTAATTCGTATCTTTCAACTGACTGGGTAGGAGATTT$'

In [252]:
get_bwt(text)

'TGATATAGGATAAGCATGATAATTATGACCGGTGTGAAACTCCTCTGATTCAGAGGCGTAGACAACGGCGAGCTCTCAGTATCGAAGCCGAGCGATCAGATCACGTAGGTTAGACTTCTGAGTTTCGTATGCAGCGAGTCGAATCTTTTTAGGCCTGGTTCAGTCCCTTCGTGGTGTGTTCGAAACCTTAGCTGGGCCCTATGTGGCACTGTAGTCGAGCTCGTTTGGGTCTTGCCAGGTGTGAGCCCCGTACTCGCGCCTTTGCAGGAGTACCGTTACCTGAGCCGTCGGCCTGTGCTCGCTCTAGGATAGCCAACCGAGCGTCAACATAAAAGGGTCTAAGCACAACAG$TAAGGCTCCTTTAAGTTCACGTAAGAGCGGCCCAATTCGCTGCTGCAAGCTACAGGGTGCAGGTGAAGCGTGCTTACTATTATAATCCGTCTGACGCTTAACCGAGCCTATTAGACAACTAATTCCGCAGGATTACTAGGGTCGTACTCGTGCGGTGCCAATTGGACTGCAACGCAAGAGTCGTAACATACCGTCCACGGGATGACACCCTCTTCAACTGATTCACGTTTCTGAGGCTGTCTGCGGTGGAAGTAGGTCGGATTACGAGATCCCTGATCAGAGAATCGTCGTTTCAGAGTGTTTCGATTAGGTGACTTAGTTAAGTTATCTGACCCTGCTGGCCAGCTAGATGCATCAAGGGTCTCCCGCTGGGAACGCTAGTGACCGTTGTTGATACGTCCATTTTCCCAACTCACAGCGGTCCAGCAAGTATAACCAGCGTGAATGGGAAATGGAGCGAATGCCTCCTCGAGAGGGCTGAGTATTCCGTCTCTCGCATACGAGTAGACTTACTTGAACTCCATTAGTCCGGAGTTTCTGGTA'

#### 2.3 Inverting the Burrows-Wheeler Transform<a name = 'inverting'></a>

Như chúng ta đã đề cập, chúng ta sẽ không sử dụng một phương pháp nén Text mà không thể giải nén. Để ý rằng cột thứ nhất của M(Text) chính là cách nén ngây thơ mà chúng ta sẽ không thể giải nén được ra chuỗi gốc. Vậy các cột còn lại thì sao? Thật may mắn, các nhà khoa học đã chứng minh được rằng cột cuối cùng của M(Text) sẽ giải nén được thành chuỗi gốc ban đầu. Và từ cột cuối cùng này ta có thể dễ dàng biết được cột đầu tiên của M(Text) bằng cách xắp xếp lại chuỗi theo thứ tự từ vựng. Để xây dựng quá trình giải nén chúng ta sẽ xem xét các tính chất của Burrows-Wheeler Transform.

#### 2.3.1 Tính chất vòng quay chu trình

Chúng ta hãy xem xét hàng đầu tiên của M(Text), kí tự $ nằm ở vị trí đầu tiên do cách sắp xếp theo thứ tự của cột đầu tiên, và ký tự này sẽ được đặt lên ngay trước kí tự thứ hai của hàng đầu tiên sau một vòng quay nào đó, vòng quay đó chính là hàng thứ tư của M(Text). Như vậy ta xác định được ký tự thứ hai của hàng đầu tiên là "a".

<a><img src="https://imgur.com/MASCoMI.jpg" width="500" align="center"> 

Thật dễ dàng để xác định ký tự "a" này, lý do là chỉ có duy nhất một ký tự "$" trong chuỗi Text. Để ý tại các hàng 7, 8, 9, 10 có đến bốn ký tự "a", vậy ta sẽ chọn "b", "c", hay "d" cho ký tự thứ ba của hàng đầu tiên? Để giải quyết vấn đề này ta hãy tìm hiểu tính chất tiếp theo.

#### 2.3.2 Tính chất đầu cuối

Chúng ta thấy rằng, cột đầu tiên của M(Text) là một chuỗi được sắp xếp theo thứ tự từ vựng mà các chữ cái giống nhau được nhóm theo một cụm. Còn với cột cuối cùng, các chữ cái hoàn toàn không được nhóm theo một cụm, ví dụ "r" ở hàng hai và hàng năm. Tuy nhiên thứ tự của các ký tự giống nhau sẽ phải được bảo toàn ở cột cuối giống như cột đầu. Tính chất này được phát biểu như sau:

<i>Sự xuất hiện lần thứ k của một kí tự trong cột đầu và lần xuất hiện thứ k của ký tự này trong cột cuối tương ứng với cùng vị trí của ký tự này trong Text.</i>

<a><img src="https://imgur.com/JMy1BPi.jpg" width="300" align="center"> 

Giả sử các ký tự giống nhau được đánh chỉ số theo thứ tự của nó thì ta sẽ dễ dàng tìm được các ký tự còn lại của hàng đầu tiên. Mặt khác, các kí tự này có thể coi như các ký tự duy nhất trong chuỗi, giống với kí tự $. Từ đó ta có thể áp dụng quá trình đảo ngược này để tìm một chuỗi của bất kì hàng nào trong ma trận M(Text).

<a><img src="https://imgur.com/8kryCH0.jpg" width="500" align="center"> 

Bây giờ ta đi tìm chuỗi gốc. Để ý rằng có duy nhất một ký tự $ được đặt vào cuối của chuỗi Text, do đó chuỗi kí tự gốc chính là chuỗi ở hàng thứ tư của M(Text).

In [26]:
class InverseBurrowsWheelerTransform(object):
    
    def __init__(self, bwt):
        self._bwt = bwt
        # Starting Positions of Patterns matching Text.
        self._start_positions = []
        # Generate first column from BWT.
        self._first_column = sorted(self._bwt)
        # Keeping last column in string type to speed up runtime.
        self._last_column_str = self._bwt
        # Last column in list type.
        self._last_column = list(self._bwt)
        # Get symbols of the first column.
        self._alphabet = np.unique(self._first_column)
        
        # Adding index to the first column.
        self._index_first_column = self.index_symbol_of_column(self._first_column)
        # Adding the same index of corresponding symbols to the last column.
        self._index_last_column = self.index_symbol_of_column(self._last_column)
        # Get index of the first symbol of the first column.
        self._first_occurence = self.get_first_occurence()
        
    # Inverting a bwt to original text
    def invert_bwt(self):
        
        org_string = []
        length = len(self._index_first_column)
        
        # Find row concluding symbol (0, '$') at last.
        index_str = self._index_last_column.index((0, '$'))
        
        while 1 < 2:
            # Finding the first symbol of the row above
            org_symbol_of_string = self._index_first_column[index_str]
            # Only choosing symbol, excluding index.
            org_string.append(org_symbol_of_string[1])
            # The loop is going on.
            index_str = self._index_last_column.index(org_symbol_of_string)
            # If the loop returns symbol (0, '$'), we stop the loop.
            if org_symbol_of_string == (0, '$'):
                break
                
        return(''.join(org_string))
    # Find the number of patterns matching text by bwt.
    def match_multi_patterns_bwt(self, patterns):
        number = []
        for pattern in patterns:
            number.append(self.match_bwt(pattern))
            
        return(number)
    # Find the number of patterns matching text by better bwt.
    def match_multi_patterns_better_bwt(self, patterns):
        number = []
        for pattern in patterns:
            number.append(self.get_better_bw_matching(pattern))
            
        return(number)
    # Find the number of patterns matching text by better bwt with checkpoints.
    def match_multi_patterns_better_bwt_with_check_points(self, patterns, c):
        number = []
        self._index_check_point, self._count_dict = self.get_check_point_arrays(c)
        for pattern in patterns:
            number.append(self.get_better_bw_matching_with_check_points(pattern))
            
        return(number)
    # Find positions of text that patterns matching the text by better bwt.
    def locate_multi_patterns_better_bwt(self, patterns, partial_suffix_array, suffix_array_index):
        positions = []
        for pattern in patterns:
            positions.append(self.locate_better_bw_matching(pattern, partial_suffix_array, suffix_array_index))
            
        return(positions)
    
    # Find positions of text that patterns matching the text by better bwt with check point.
    def locate_multi_patterns_better_bwt_with_check_points(self, patterns, c, partial_suffix_array, suffix_array_index):
        positions = []
        self._index_check_point, self._count_dict = self.get_check_point_arrays(c)
        for pattern in patterns:
            positions.append(self.locate_better_bw_matching_with_check_points(pattern, c, partial_suffix_array, suffix_array_index))
            
        return(positions)
    # Find the number of pattern matching text by bwt.
    def match_bwt(self, pattern):
        # Set the initial top and bottom.
        top = 0
        l = len(self._last_column_str)
        bottom = l - 1
        
        while top <= bottom:
            if pattern != '':
                symbol = pattern[-1] # Moving backward of Pattern.
                pattern = pattern[0 : -1] # Remove the last symbol of pattern.
               
                if symbol in self._last_column_str[top : (bottom + 1)]:
                    # Finding top_index and bottom_index of next symbol at the last column.
                    top_index = self._last_column_str[top : (bottom + 1)].index(symbol) + top
                    bottom_index = bottom - list(reversed(self._last_column_str[top : (bottom + 1)])).index(symbol)
                    # Updating top and bottom.
                    top = self.get_last_to_first(top_index)
                    bottom = self.get_last_to_first(bottom_index)
                    
                else:
                    return(0)
            else:
                # Return number of positions of text that pattern matches text at that positions.
                return(bottom - top + 1) 
    # Find the number of pattern matching text by better bwt.
    def get_better_bw_matching(self, pattern):
        # Set the initial top and bottom.
        top = 0
        bottom = len(self._last_column_str) - 1
        
        while top <= bottom:
            if pattern != '':
                symbol = pattern[-1] # Moving backward of Pattern.
                pattern = pattern[0 : -1] # Remove the last symbol of pattern.
                # Updating top and bottom.
                top = self._first_occurence[symbol] + self.count_symbol(symbol, top)
                bottom = self._first_occurence[symbol] + self.count_symbol(symbol, bottom + 1) - 1
            else:
                return(bottom - top + 1)
        return(0)
    # Find the number of pattern matching text by better bwt with checkpoints.
    def get_better_bw_matching_with_check_points(self, pattern):
        # Set the initial top and bottom.
        top = 0
        bottom = len(self._last_column_str) - 1
        
        while top <= bottom:
            
            if pattern != '':
                symbol = pattern[-1] # Moving backward of Pattern.
                pattern = pattern[0 : -1] # Remove the last symbol of pattern.
                # Updating top and bottom with checkpoints.
                if top not in self._index_check_point:
                    top = self._first_occurence[symbol] + self.count_symbol(symbol, top)
                else:
                    top = self._first_occurence[symbol] + self._count_dict[top][symbol]

                if (bottom + 1) not in self._index_check_point:
                    bottom = self._first_occurence[symbol] + self.count_symbol(symbol, bottom + 1) - 1
                else:
                    bottom = self._first_occurence[symbol] + self._count_dict[bottom + 1][symbol] - 1

            else:
                return(bottom - top + 1)
        return(0)
   
    # Find positions of text that pattern matching the text by better bwt.
    def locate_better_bw_matching(self, pattern, partial_suffix_array, suffix_array_index):
        # Set the initial top and bottom.
        top = 0
        bottom = len(self._last_column_str) - 1
        starting_positions = []
      
        while top <= bottom:
            if pattern != '':
                symbol = pattern[-1] # Moving backward of Pattern.
                pattern = pattern[0 : -1] # Remove the last symbol of pattern.
                # Updating top and bottom.
                top = self._first_occurence[symbol] + self.count_symbol(symbol, top)
                bottom = self._first_occurence[symbol] + self.count_symbol(symbol, bottom + 1) - 1
            else:
                # Continuously backward to find the locations.
                for i in range(top, bottom + 1):
    
                    count = 0
                    last_index = i
                    
                    while last_index not in suffix_array_index:
                        
                        count += 1
                        symbol = self._last_column[last_index]
                        last_index = self._first_occurence[symbol] + self.count_symbol(symbol, last_index)
                      
                    starting_positions.append(partial_suffix_array[last_index] + count)
                    
                return(sorted(starting_positions))
            
        return('None')
    # Get approximatly patterns matching text by better bwt.
    def get_approximate_pattern_matching_with_better_bwt(self, patterns, d, partial_suffix_array, suffix_array_index):
        
        starting_positions = []
        
        for pattern in patterns:
            
            self._start_positions = []
            # Set initial top and bottom.
            top = 0
            l = len(self._last_column)
            bottom = l - 1
            
            """ If we want to assign the initial value of the pattern to match the symbol then run the hidden code folows:"""
            #symbol = pattern[-1]
            #pattern = pattern[0 : -1]

            #if symbol in self._last_column[top : (bottom + 1)]:

                #top_index = self._last_column[top : (bottom + 1)].index(symbol) + top
                #bottom_index = bottom - list(reversed(self._last_column[top : (bottom + 1)])).index(symbol)
                #top = self.get_last_to_first(top_index)
                #bottom = self.get_last_to_first(bottom_index)
                
            starting_positions.append(self.get_sub_aproximate_string(pattern, top, bottom, d, 0, 
                                                                     partial_suffix_array, suffix_array_index)
                                     )
        return(starting_positions)
    # Get approximatly patterns matching text by better bwt with checkpoints.
    def get_approximate_pattern_matching_with_check_points(self, patterns, c, d, partial_suffix_array, suffix_array_index):
        
        starting_positions = []
        self._index_check_point, self._count_dict = self.get_check_point_arrays(c)
      
        for pattern in patterns:
            self._start_positions = []
            # Set initial top and bottom.
            top = 0
            l = len(self._last_column)
            bottom = l - 1
            
            """ If we want to assign the initial value of the pattern to match the symbol then run the hidden code folows:"""
            #symbol = pattern[-1]
            #pattern = pattern[0 : -1]

            #if symbol in self._last_column[top : (bottom + 1)]:

                #top_index = self._last_column[top : (bottom + 1)].index(symbol) + top
                #bottom_index = bottom - list(reversed(self._last_column[top : (bottom + 1)])).index(symbol)
                #top = self.get_last_to_first(top_index)
                #bottom = self.get_last_to_first(bottom_index)
                
            starting_positions.append(self.get_sub_aproximate_string_with_check_points(pattern, top, bottom, d, 0, 
                                                                                       partial_suffix_array, suffix_array_index)
                                     )
        return(starting_positions)
    # Get approximatly pattern matching text by better bwt.
    def get_sub_aproximate_string(self, pattern, top, bottom, d, initial_mismatch, 
                                  partial_suffix_array, suffix_array_index
                                 ):
        # Get symbol exiting from top to bottom of the last column.
        apr_symbol_alphabet = ''.join(np.unique(self._last_column[top : (bottom + 1)]))
        # If pattern equals '', we find the positions of text that pattern matches the text.
        if pattern == '':
            
            for i in range(top, bottom + 1):
                # Set the initial backward count.
                count = 0
                last_index = i
                # When partial suffix array is not exist, we go backward again.
                while last_index not in suffix_array_index:
                    
                    count += 1
                    symbol = self._last_column_str[last_index]
                    last_index = self._first_occurence[symbol] + self.count_symbol(symbol, last_index)
            # Starting position of pattern equals sum of partial suffix array and extend backward count.        
            self._start_positions.append(partial_suffix_array[last_index] + count)
            
        else:
            symbol = pattern[-1] # Moving backward of Pattern.
            pattern = pattern[:-1] # Remove the last symbol of pattern.
            for apr_symbol in apr_symbol_alphabet:
                # Set the initial mismatch.
                mismatch = initial_mismatch
                
                if apr_symbol != symbol:
                    mismatch += 1
                # If mismatch <= d, we update top and bottom.
                if mismatch <= d:   
                    
                    new_top = top
                    new_bottom = bottom
                    
                    """ If we want to use normal bwt instead of better bwt then run the hidden code folows:"""
                    #top_index = self._last_column[new_top : (new_bottom + 1)].index(apr_symbol) + new_top
                    #bottom_index = new_bottom - list(reversed(self._last_column[new_top : (new_bottom + 1)])).index(apr_symbol)
                    #new_top = self.get_last_to_first(top_index)
                    #new_bottom = self.get_last_to_first(bottom_index)
                    
                    new_top = self._first_occurence[apr_symbol] + self.count_symbol(apr_symbol, new_top)
                    new_bottom = self._first_occurence[apr_symbol] + self.count_symbol(apr_symbol, new_bottom + 1) - 1
                    # Using recurence algorithm for the new patterns that are exact or in-exact matching text.
                    self.get_sub_aproximate_string(pattern, new_top, new_bottom, d, 
                                                                    mismatch, partial_suffix_array, suffix_array_index
                                                                    ) 
        return(self._start_positions)
    # Get approximatly pattern matching text by better bwt with checkpoints.
    def get_sub_aproximate_string_with_check_points(self, pattern, top, bottom, d, initial_mismatch, 
                                  partial_suffix_array, suffix_array_index
                                 ):
        # Get symbol exiting from top to bottom of the last column.
        apr_symbol_alphabet = ''.join(np.unique(self._last_column[top : (bottom + 1)]))
        # If pattern equals '', we find the positions of text that pattern matches the text.
        if pattern == '':
            
            for i in range(top, bottom + 1):
                # Set the initial backward count.
                count = 0
                last_index = i

                while last_index not in suffix_array_index:

                    count += 1
                    symbol = self._last_column_str[last_index]
                    last_index = self._first_occurence[symbol] + self.count_symbol(symbol, last_index)
                    
            self._start_positions.append(partial_suffix_array[last_index] + count)
            
        else:
            symbol = pattern[-1]
            pattern = pattern[:-1]
            for apr_symbol in apr_symbol_alphabet:
                # Set the initial mismatch.
                mismatch = initial_mismatch
                
                if apr_symbol != symbol:
                    mismatch += 1
                # If mismatch <= d, we update top and bottom.
                if mismatch <= d:   
                    
                    new_top = top
                    new_bottom = bottom
                    
                    """ If we want to use normal bwt instead of better bwt then run the hidden code folows:"""
                    #top_index = self._last_column[new_top : (new_bottom + 1)].index(apr_symbol) + new_top
                    #bottom_index = new_bottom - list(reversed(self._last_column[new_top : (new_bottom + 1)])).index(apr_symbol)
                    #new_top = self.get_last_to_first(top_index)
                    #new_bottom = self.get_last_to_first(bottom_index)
                    
                    #new_top = self._first_occurence[apr_symbol] + self.count_symbol(apr_symbol, new_top)
                    #new_bottom = self._first_occurence[apr_symbol] + self.count_symbol(apr_symbol, new_bottom + 1) - 1
                   
                    # Updating top and bottom using checkpoints.
                    if new_top not in self._index_check_point:
                        new_top = self._first_occurence[apr_symbol] + self.count_symbol(apr_symbol, new_top)
                    else:
                        new_top = self._first_occurence[apr_symbol] + self._count_dict[new_top][apr_symbol]
                    
                    if new_bottom + 1 not in self._index_check_point:
                        new_bottom = self._first_occurence[apr_symbol] + self.count_symbol(apr_symbol, new_bottom + 1) - 1 
                    else:
                        new_bottom = self._first_occurence[apr_symbol] + self._count_dict[new_bottom + 1][apr_symbol] - 1
                     # Using recurence algorithm for the new patterns that are exact or in-exact matching text.
                    self.get_sub_aproximate_string(pattern, new_top, new_bottom, d, 
                                                                    mismatch, partial_suffix_array, suffix_array_index
                                                                    ) 
        return(self._start_positions)
    # Find positions of text that pattern matches the text.
    def locate_better_bw_matching_with_check_points(self, pattern, c, partial_suffix_array, suffix_array_index):
        # Set the initial top and bottom
        top = 0
        bottom = len(self._last_column_str) - 1
        starting_positions = []
        
        while top <= bottom:
            if pattern != '':
                symbol = pattern[-1] # Go backward through the pattern.
                pattern = pattern[0 : -1] # Remove the last symbol of pattern.
                # Updating top and bottom with checkpoints.
                if top not in self._index_check_point:
                    top = self._first_occurence[symbol] + self.count_symbol(symbol, top)
                else:
                    top = self._first_occurence[symbol] + self._count_dict[top][symbol]
                    
                if bottom + 1 not in self._index_check_point:
                    bottom = self._first_occurence[symbol] + self.count_symbol(symbol, bottom + 1) - 1
                else:
                    bottom = self._first_occurence[symbol] + self._count_dict[bottom + 1][symbol] - 1
            else:

                for i in range(top, bottom + 1):
                    # Set the initial extend backward count.
                    count = 0
                    last_index = i
                    # If partial suffix array doesn't exist, we go backward again.
                    while last_index not in suffix_array_index:

                        count += 1
                        symbol = self._last_column_str[last_index]
                        last_index = self._first_occurence[symbol] + self.count_symbol(symbol, last_index)

                    starting_positions.append(partial_suffix_array[last_index] + count)

                return(sorted(starting_positions))

        return('None')
    # Adding index for each symbol.
    def index_symbol_of_column(self, column):
       
        l = len(column)
        col = column[::]
        
        for x in self._alphabet:
            count = 0
            for i in range(l):
                # Index equals occurence order of symbol.
                if col[i] == x:
                    count = count + 1
                    col[i] = (count, x)
                    
        return(col) 
    # Finding index of symbol in the first column coresponds to the same symbol in the last column.
    def get_last_to_first(self, last_index):
        
        first_index = self._index_first_column.index(self._index_last_column[last_index])
        
        return (first_index)
    # Finding index of the symbol occuring at first in the first column.
    def get_first_occurence(self):
        first_occurence = {}
        for x in self._alphabet:
            first_occurence.update({x : self._first_column.index(x)})
        return(first_occurence)
    # Count symbol from 0 to index.
    def count_symbol(self, symbol, index):
        
        return(self._last_column_str.count(symbol, 0, index))
    # Construct checkpoints arrays.
    def get_check_point_arrays(self, c):
        
        length = len(self._last_column)
        index_check_point = np.arange(0, length + 1, c).tolist()
        count_dict = {}

        for i in index_check_point: 
            check_point_arrays = {}
            instance_column = self._last_column[0: i]
            # Finding number of symbols' occurence from 0 to i.
            count_dict.update({i:Counter(instance_column)})
        
        return(index_check_point, count_dict)   

    def divide_pattern(self, pattern, d):
        sub_strings = []
        k = int(len(pattern)/(d+1))
        for i in range(0, d):
            sub_strings.append(pattern[i*k: ((i+1)*k)])
        sub_strings.append(pattern[d*k::])
        return(sub_strings)

In [675]:
bwt = 'TGCTCAGGCAATCTCATAGAAGATCTATGATTAAACTGTCTCGGGAGGAAGTCGTGGCTACATCACATAAGATGGAATAAACAGTATACATACCGCCAGTTCGGCAATGCTGTCGACTTGAAGCTGCGTCATGGCCAAGACAACTTCGTGCAACGTTGATGACTACTTGAAGGGACCCAACGCTAGAGGCAGTGCTCTCGGTACTGGTAATCCACGAATCAAAAATGCCTGATTGCGCGAGTGTTTAGTGGGACCCATGAGTAATTTCAGTGAACGACACGCTGATGTACTAGTTCTTCCCTAAATTACGAGCGAACCTATAGAGTACGTTGCCTTGGTGACCGCCTGCGGGCACAACGATAGTATGTTTGCAGAAACTGCGACCGTGGCAGATAGCGAACAGGCCAGGTGCTCAGATGCCACTCGAAGTCGCTGCACTAAGTCACGACCTCCGGCGGCAACGCTGCACTTGGCTAGTCACTGGCACATCTAGTCGGAAAGGATATTCACTACACTAGAATCCATGATCGCCCAGCCGTAACGGTCTGGAGGACCGCAGTGCTACTTTGTGTAAATACTAAAATCTGCCAAGGGTCGGCGAATTCTCGTAGAGGGGGCCGGGTAATGGCCTAATCTGCGCCTTAGCTGTCATTGCTGTTGGCGAGTTAAACTATTCTTACATCTTGTACATCAGCTTTTGAGGGGCGATGTATTCTAATCTATCCTGTGCGTCTCCTCACTGCGCAGATCGTATATCGTCTCGGTACTATGCATCCGTGTTACCCGTCTGAAGATAATGGCGAGACACTT$AACCAGCATGGTGCGTCCCGGCCATGGCCGTGCACAACGAGGTATCTCCGAAGTAGACAGTCAAATGCGTACTTTCCTCTAATATTGCGTGGCGAAGTATAGTGAATTTAGATGAGATAGTTACAGCTCGGGAGGGTAATCCGTAAGGCCTC'

In [627]:
InverseBurrowsWheelerTransform(bwt).invert_bwt()

'TCCTGTGACGGGCAGGAACACTGCTGCCGGGCAATCGAAGACGGACAAGGGACGCACGTGCCTGGGTGTACACCCAGATCAGCTATGGACAGTACCAGTTTATTGGCGAAGAGCGCAAAGAGGCGAGTTAATGCACGCTACGATGCCTCGCCTAAATTCTCCGCGTTAAAACAGAGGGGCAGCAGTAGAGGGTTAACACTCCACTTTTAGCGCCCTGAAATGGTAACACCAAGACCTTAAACCGTCGATTAAGAAAATGGGCTGACTCTGAAAAATCATGAAGCCGATGTGGAACGGTGCCTAGTGAGTCTTGGTGTTGAAGTAAAGCGTAGCGGGCCGGGTGTTTGCGAGAAGTCATTCCGGGGGCCCTAACGATCCCTGCGTTTGTAGCTCCGAGCAAACCATTATTTACCTGCAAAACACGAGCGCTAGCGCCTCGAAGCTTTTTCATCATGCAGACCCGCCATGTTAGAGTTGTCTCAAGTTCATGGCTCTATCCCGATTGCGCGGACGAAATCGACTGGGCAATGGGATTGTGCCTTTATTACTGTTGTCAGTCCCCCATGTGGAGGGATCCGCCCGTGAGACTAATTTGAGCGGTTAGGATACGTTGGGGTCTAGAAGAGAAGCCATGATCAGTACGCTGCGGCCAGCCTCAAAAGCGGGCCGGCATATTTACTAACATATAGGATTCACCCTATGTATTAACCAACTATTTCTAACTACAGTGAATTACGGCTGTAATGGTAAACCAGAGATTATATCGAAAATGGAAGTGGTGTTGTGGCTTTATCGCTTGTCACGGCTGTCGTTGTTCACCCCTCTGTCGTCTTTGTTGTTCCTAGCAAAATGGTGGGCTGATACCAACGAGTTGTCCCTAAGGCTAATAAGTATCATACTCGACAACAATGCGATTCGTCACAGACCTCTTGCGTCCACCTGTATTATACCTGACTCACCCT$'

#### 2.4 Pattern Matching with the Burrows-Wheeler Transform<a name = 'pattern-matching'></a>

#### 2.4.1 A first attempt at Burrows-Wheeler pattern matching

Chúng ta đã biết có thể xây dựng lại ma trận M(Text) mà chỉ cần biết duy nhất cột cuối cùng của nó hay còn gọi là Burrows-Wheeler Transform. Để ý rằng ma trận M(Text) có các hàng chứa các Suffix mà chạy từ điểm đầu tiên đến vị trí của $. Tuy nhiên chúng ta không thể lưu trữ toàn bộ M(Text) nếu Text quá lớn như bộ gen người. Do đó, chúng ta sẽ tìm cách hay vì lưu lại toàn bộ M(Text) ta chỉ cần lưu vị trí bắt đầu của Suffix trong Text, hay chính là Suffix Array. Dựa vào Burrows-Wheeler Transform của M(Text) ta có thể xây dựng được thuật toán khớp một Pattern vào Text.

<a><img src="https://imgur.com/iRsz1u5.jpg" width="500" align="center"> 

#### 2.4.2 Moving backward through a pattern

Như đã biết, từ cột Burrows-Wheeler Transform ta có thể xây dựng được cột đầu tiên của M(Text), từ hai cột nay ta xây dựng thuật toán khớp một Pattern vào Text bằng cách di chuyển lùi qua Pattern. Ví dụ, nếu chúng ta muốn khớp mẫu = "ana" với Text = "panamabananas$", trước tiên chúng ta sẽ xác định các hàng của M (Text) bắt đầu bằng "a", chữ cái cuối cùng của "ana":

<a><img src="https://imgur.com/9W9CQgs.jpg" width="300" align="center"> 

Khi chúng ta đang di chuyển ngược qua "ana", chúng ta sẽ tìm các hàng của M (Text) bắt đầu bằng "na". Để làm điều này mà không cần biết toàn bộ ma trận M (Text), thực tế là một kí tự trong cột cuối (LastColumn) phải đứng trước kí tự của Text được tìm thấy trong cùng một hàng của cột đầu (FirstColumn). Vì vậy, chúng ta chỉ cần xác định các hàng M (Text) bắt đầu bằng "a" và kết thúc bằng "n":

<a><img src="https://imgur.com/R3GyQ5a.jpg" width="300" align="center"> 

Tính chất đầu cuối cho chúng ta biết nơi tìm thấy ba chữ "n" được tô sáng trong FirstColumn, như hình dưới đây tất cả ba hàng kết thúc bằng "a", mang lại ba lần xuất hiện của "ana" trong Text.

<a><img src="https://imgur.com/6XwP7jS.jpg" width="300" align="center"> 

Sự xuất hiện nhiều lần của "a" trong LastColumn tương ứng với lần thứ ba, thứ tư và lần xuất hiện thứ năm của "a" trong cột này, tính chất đầu cuối cho chúng ta biết rằng chúng phải tương ứng với lần xuất hiện thứ ba, thứ tư và thứ năm của "a" trong FirstColumn, vì vậy chúng ta có thể xác định được ba vị trí khớp của của "ana":

<a><img src="https://imgur.com/1SBWsQP.jpg" width="300" align="center"> 

#### 2.4.3 The Last-to-First mapping

Bây giờ chúng ta đã biết cách sử dụng BWT (Text) để tìm tất cả các khớp của Pattern trong Text bằng cách quay lui qua pattern. Tuy nhiên, mỗi khi chúng ta quay lui, chúng ta cần phải theo dõi các hàng của M (Text) nơi chứa các khớp của hậu tố Pattern. Chúng ta biết rằng ở mỗi bước các hàng M (Text) khớp với một hậu tố của Pattern gom lại với nhau trong các hàng liên tiếp của M (Text). Điều này có nghĩa là tập hợp của tất cả các hàng khớp chỉ cần biểu diễn bởi hai con trỏ, trên cùng (top) và dưới cùng (bottom): top là chỉ mục của hàng đầu tiên của M (Text) khớp với hậu tố hiện tại của Pattern và bottom là chỉ mục của hàng cuối cùng của M (Text) khớp với hậu tố này. Hình dưới đây mô tả sự thay đổi của top và bottom trong quá trình khớp Pattern = "ana".

<a><img src="https://imgur.com/2rfXZiu.jpg" width="800" align="center"> 

Để xác định các top và bottom ta cần thực hiện vòng lặp trong quá trình quay lui Pattern như sau:<br><br>
topindex $\gets$ vị trí đầu tiên của kí tự thuộc Pattern match với một kí tự ở cột cuối giữa các vị trí top và bottom ở cột cuối<br>
bottomindex $\gets$ vị trí cuối cùng của kí tự thuộc Pattern match với một kí tự ở cột cuối giữa các vị trí top và bottom ở cột cuối<br>
top $\gets$ LasttoFirst (topindex)<br>
bottom $\gets$ Lasttofirst (bottomindex)<br>

Trong bảng dưới đây, cột Last to First (i) cho thấy giá trị của các top và bottom ở các bước tiếp theo sau khi đã xác định được vị trí của các kí tự ở cột cuối của M(Text) của bước trước đó. Ta cũng có thể tính được số lần khớp của Pattern bằng bottom - top + 1. Cột Count đếm số kí tự xuất hiện từ đầu tiên cho đến trước các vị trí ở cột Last to First (i). Chúng ta sẽ thấy các giá trị của cột Count tham gia vào thuật toán ở phần sau.

<a><img src="https://imgur.com/d8Lxz3R.jpg" width="800" align="center"> 

In [66]:
with open('D:/Data Science/Data/Mutations/How Do We Locate Disease/Multi_patterns_bwt/dataset_301_7.txt') as f:
    lines = [line.rstrip() for line in f]
    f.close
bwt = lines[0]
patterns = lines[1].split(' ')

In [69]:
import time
print('Length of bwt: ', len(bwt))
print('Number of patterns: ', len(patterns))
print('Number of substring matches of the i-th member of patterns in text: ')
start_time = time.time()
print(InverseBurrowsWheelerTransform(bwt).match_multi_patterns_bwt(patterns))
print("--- %s seconds ---" % (time.time() - start_time))

Length of bwt:  20001
Number of patterns:  3915
Number of substring matches of the i-th member of patterns in text: 
[1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 

#### 2.5 Speeding Up Burrows-Wheeler Pattern Matching<a name = 'speed-up'></a>

Ở phần trước, khi đi tìm các top và bottom ta cần so sánh các topindex và bottomindex với các kí tự ở cột đầu tiên, sự so sánh này sẽ làm tăng thời gian tính toán đồng thời làm tăng yêu cầu của bộ nhớ do phải lưu tất cả các kí tự ở cột đầu tiên. Giải pháp để làm giảm chi phí ở đây là chúng ta chỉ lưu kí tự và vị trí đầu tiên của nó ở cột đầu tiên, ta có thể xây dựng hàm $FirstOccurence (symbol)$ để tìm vị trí đầu tiên của kí tự symbol ở cột đầu tiên. Dựa vào tính chất đầu cuối, vị trí của symbol ở cột đầu ở bước sau sẽ được tính bằng $FirstOccurence (symbol)$ cộng với số lần xuất hiện symbol trước vị trí của nó ở cột cuối của bước trước. Số lần xuất hiện này có thể được tính bằng hàm $Count(symbol, i)$. $$top \gets FirstOccurence(Symbol) + Count(symbol, top)\\
bottom \gets FirstOccurence(Symbol) + Count(symbol, bottom + 1) - 1$$

In [82]:
import time
print('Length of bwt: ', len(bwt))
print('Number of patterns: ', len(patterns))
print('Number of substring matches of the i-th member of patterns in text: ')
InverseBurrowsWheelerTransform(bwt).match_multi_patterns_better_bwt(patterns)
print("--- %s seconds ---" % (time.time() - start_time))

Length of bwt:  20001
Number of patterns:  3915
Number of substring matches of the i-th member of patterns in text: 
--- 2.6984572410583496 seconds ---


Mặc dù thuật toán của chúng ta là tăng tốc Burrows-Wheeler Pattern Matching, nhưng có vẻ ta vẫn còn có thể làm cho thuật toán nhanh hơn. Lý do ở đây là chúng ta đang phải tính số lần xuất hiện của symbol sau mỗi vòng lặp bằng hàm $Count(symbol, i)$. Để làm cho runtime nhanh hơn ta có thể tính trước số lần xuất hiện của symbol như cột Count ở phần trước, sau đó chỉ cần gọi lại kết quả này. Tuy nhiên, cách làm này lại làm tăng yêu cầu lưu trữ mà chúng ta đang muốn giảm đi. Do đó chúng ta mới gọi thuật toán này là BetterBWTMatching. Một cách cân đối giữa runtime và storage là chỉ lưu một phần cột Count này. 

<i>Chúng ta sẽ chỉ lưu trữ mảng Count (Count arrays) khi i chia hết cho C, trong đó C là hằng số; những mảng này được gọi là checkpoint arrays. Khi C lớn (trên thực tế C thường bằng 100) và bảng chữ cái là nhỏ (ví dụ: bốn nucleotide), check point arrays yêu cầu chỉ một phần bộ nhớ được sử dụng bởi BWT (Text).</i>

Còn thời gian chạy thì sao? Sử dụng checkpoint arrays, chúng ta có thể tính toán con trỏ top và bottom trong một số bước không đổi. Vì mỗi chuỗi Pattern yêu cầu nhiều nhất |Pattern| cập nhật con trỏ, thuật toán BetterBWTMatching đã sửa đổi hiện yêu cầu thời gian chạy $\mathcal{O} (|Patterns|)$, giống như sử dụng Trie hoặc Suffix Array.

In [95]:
import time
print('Length of bwt: ', len(bwt))
print('Number of patterns: ', len(patterns))
print('Number of substring matches of the i-th member of patterns in text: ')
start_time = time.time()
ki = InverseBurrowsWheelerTransform(bwt).match_multi_patterns_better_bwt_with_check_points(patterns, 500)
print("--- %s seconds ---" % (time.time() - start_time))

Length of bwt:  20001
Number of patterns:  3915
Number of substring matches of the i-th member of patterns in text: 
--- 2.589503049850464 seconds ---


#### 2.6 Locating Matched Patterns with BWT<a name = 'locate-bwt'></a>

#### 2.6.1 Partial Suffix Array

Để xác định vị trí khớp Pattern, một lần nữa chúng ta có thể sử dụng Suffix Array. Trong phần 2.4.1, Suffix Arrays ngay lập tức tìm thấy ba vị trí của "ana" trong "panamabanana $".
Suffix Arrays làm cho công việc của chúng ta dễ dàng, nhưng nhớ lại rằng động lực ban đầu của chúng ta là sử dụng biến đổi Burrows-Wheeler là để giảm yêu cầu về bộ nhớ được sử dụng bởi Suffix Array cho việc khớp Pattern. Nếu chúng ta thêm Suffix Array vào thuật toán khớp Pattern dựa trên Burrows-Wheeler, thì chúng ta sẽ trở lại ngay nơi bắt đầu.

Chúng tôi sẽ xây dựng một Partial Suffixx Array của Text, ký hiệu là SuffixArray (Text, k), chỉ chứa các giá trị là bội số của một số nguyên dương k. Trong các thực tế, Partial Suffix Array thường được xây dựng cho k = 100, do đó giảm sử dụng bộ nhớ so với Suffix Array đầy đủ. 

Trong Partial Suffix Array ta chỉ lưu các dãy cách nhau k đơn vị. Do đó, khi tìm được một Pattern khớp với Text là "ana", ta sẽ dừng việc quay lui nếu tương ứng với hàng chứa Pattern = "ana" tồn tại dãy hậu tố một phần. Ngược lại, nếu chưa có Partial Suffix Array tương ứng ta tiếp tục quay lui.

<a><img src="https://imgur.com/Qp9cVjR.jpg" width="800" align="center"> 

In [3]:
def construct_partial_suffix_arrays(text, k):

    text_patterns = []
    partial_index = []
    partial_array = []
    l = len(text)
    # Append all suffix array
    for i in range(l):
        text_patterns.append(text[i::])
    # Find index of suffix array after sorting
    suffix_array = np.argsort(text_patterns).tolist()

    array_index = np.arange(0, l, 1)
    frac = int((l-1)/k)
    
    for c in range(0, frac + 1):
        # The distance between the positions of the suffix is k.
        partial_index.append(array_index[suffix_array.index(c*k)])
        partial_array.append(c*k)
    # Sorting tuple of suffix array in order of partial index.
    tuple_suffix_array = sorted(tuple(zip(partial_index, partial_array)))
    
    partial_suffix_array = dict(tuple_suffix_array)
    suffix_array_index = list(partial_suffix_array.keys())

    return(partial_suffix_array, suffix_array_index)

#### 2.6.2 Comparing Runtime 

Chúng ta thử so sánh thời gian chạy của thuật toán xác định vị trí khớp của Pattern trong Text với các hệ số k khác nhau của Suffix Array. Với k = 1, tất cả các Partial Suffix Array trở thành Suffix Array, khi đó ta không phải quay lui bất kì Pattern nào sau khi đã khớp với Text, thời gian chạy rõ ràng là nhanh nhất nhưng yêu cầu về bộ nhớ sẽ cao nhất.

In [114]:
with open('D:/Data Science/Data/Mutations/How Do We Locate Disease/Burrows and Wheeler Set Up Checkpoints/dataset_303_4 (5).txt') as f:
    lines = [line.rstrip() for line in f]
    f.close
text = lines[0] + '$'
patterns = lines[1::]
partial_suffix_array, suffix_array_index = construct_partial_suffix_arrays(text, 1)
bwt = get_bwt(text)

import time
print('Length of bwt: ', len(bwt))
print('Number of patterns: ', len(patterns))
start_time = time.time()
positions = InverseBurrowsWheelerTransform(bwt).locate_multi_patterns_better_bwt(patterns, partial_suffix_array, suffix_array_index)
print("--- %s seconds ---" % (time.time() - start_time))

Length of bwt:  8241
Number of patterns:  15134
--- 3.6338999271392822 seconds ---


Với k = 100, thời gian chạy sẽ lâu hơn, ngược lại yêu cầu về bộ nhớ lại thấp hơn.

In [115]:
partial_suffix_array, suffix_array_index = construct_partial_suffix_arrays(text, 100)
start_time = time.time()
positions = InverseBurrowsWheelerTransform(bwt).locate_multi_patterns_better_bwt(patterns, partial_suffix_array, suffix_array_index)
print("--- %s seconds ---" % (time.time() - start_time))

--- 4.758271217346191 seconds ---


In [116]:
print('All starting positions in Text where a string from Patterns appears as a substring: ')
print(positions[0:100], "...")

All starting positions in Text where a string from Patterns appears as a substring: 
['None', 'None', 'None', 'None', 'None', [5074], 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', [2344], 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', [2188], 'None', 'None', 'None', 'None', [1008], 'None', 'None', [3115], 'None', 'None', 'None', 'None', [2508, 7391], 'None', [2796, 2803, 5808], 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', [2197], 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None'] ...


In [117]:
while 'None' in positions:
    positions.remove('None')
result = []
for x in positions:
    result += x
print(sorted(result))

[2, 11, 12, 19, 20, 21, 22, 23, 24, 25, 26, 27, 36, 59, 60, 61, 62, 63, 64, 65, 66, 75, 76, 77, 78, 81, 87, 88, 97, 98, 101, 108, 124, 125, 130, 143, 144, 145, 146, 147, 148, 150, 151, 152, 153, 154, 155, 157, 166, 167, 168, 169, 170, 171, 172, 173, 185, 193, 202, 203, 204, 205, 206, 207, 208, 209, 217, 224, 225, 226, 235, 237, 238, 239, 240, 241, 242, 252, 265, 269, 272, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 321, 323, 324, 325, 340, 341, 352, 353, 354, 355, 356, 357, 358, 359, 373, 374, 382, 383, 384, 394, 395, 409, 410, 422, 423, 424, 425, 426, 428, 429, 430, 448, 451, 460, 469, 470, 472, 473, 474, 475, 476, 482, 493, 494, 495, 496, 497, 499, 500, 510, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 538, 539, 543, 548, 549, 552, 553, 562, 571, 572, 573, 574, 575, 576, 577, 593, 594, 595, 596, 597, 598, 608, 609, 610, 611, 612, 613, 614, 615, 632, 642, 647, 652, 664, 665, 666, 667, 668, 669, 670, 671, 686, 688, 689, 690, 691, 692, 700, 701, 702, 711, 712, 713, 714, 

Với thuật toán có sử dụng checkpoint arrays, runtime sẽ nhanh hơn với một hệ số C hợp lý, ở đây ta thử với C = 50.

In [122]:
patterns = lines[1:-1]
partial_suffix_array, suffix_array_index = construct_partial_suffix_arrays(text, 50)
start_time = time.time()
positions = InverseBurrowsWheelerTransform(bwt).locate_multi_patterns_better_bwt_with_check_points(patterns, 50, partial_suffix_array, suffix_array_index)
print("--- %s seconds ---" % (time.time() - start_time))

--- 4.482427597045898 seconds ---


Bây giờ chúng ta chỉ phải lưu trữ những dữ liệu sau vào bộ nhớ: BWT (Text), FirstOccurence, Partial Suffix Array và Checkpoint Array. Lưu trữ dữ liệu này yêu cầu bộ nhớ xấp xỉ bằng 1,5 · |Text|. Vì vậy, cuối cùng chúng ta đã giảm được bộ nhớ cần thiết để giải quyết bài toán Multiple Pattern Matching cho hàng triệu sequencing reads.

#### 2.7 Approximate pattern matching with the Burrows-Wheeler transform <a name = 'approximate'></a>

Chúng ta có thể biết đến Basic Local Alignment Search Tool (BLAST) dựa trên thuật toán tham lam có thể tìm thấy sự tương đồng giữa các protein ngay cả khi tất cả các axit amin trong một protein đã bị đột biến so với protein khác [16]. Tuy nhiên ở đây, chúng ta sẽ tập trung vào phương pháp liên quan đến Burrows-Wheeler Transform.

Để mở rộng Burrows-Wheeler Transform cho khớp Pattern gần đúng, chúng ta sẽ không dừng lại khi gặp phải một mismatch. Ngược lại, chúng ta sẽ tiếp tục cho đến khi hoặc tìm thấy một kết quả gần đúng hoặc vượt quá giới hạn d mismatch [17].

Hình dưới đây minh họa việc tìm kiếm "asa" trong "panamabananas$" với nhiều nhất là 1
mismatch. Trước tiên, hãy tiến hành như trong trường hợp khớp mẫu chính xác, xâu ngược chuỗi "asa"
bằng cách sử dụng biến đổi Burrows-Wheeler. Sau khi tìm thấy sáu lần xuất hiện của "a",
chúng ta xác định sáu lần xuất hiện không chính xác của "sa" là: "pa", "ma", "ba" và ba lần xuất hiện của"na". Lưu ý rằng ba chuỗi này đã tích lũy một mismatch và sau đó tiếp tục
qúa trình tương tự với tất cả sáu chuỗi.

Trong bước tiếp theo, năm trong số các lần xuất hiện mismatch của "sa" có thể được mở rộng thành số lần xuất hiện của "asa" chỉ với một mismatch duy nhất: "ama", "aba" và ba lần xuất hiện
của "ana". Chúng ta không mở rộng "pa", mà loại bỏ vì mismatch vượt quá 1.

Trong thực tế, phương pháp tham lam này phải đối mặt với một số vấn đề phức tạp. Chúng ta không muốn giai đoạn đầu có các chuỗi không khớp, ngược lại chúng ta sẽ phải xem xét rất nhiều chuỗi không phù hợp. Do đó, chúng ta có thể chỉ xem xét các hậu tố của Pattern với một ngưỡng độ dài mà hậu tố đó khớp chính xác với Text. Hơn nữa, phương pháp này sẽ tiêu tốn nhiều thời gian khi giá trị của d lớn, vì chúng ta phải tìm kiếm nhiều khớp không chính xác. Trên thực tế, người ta thường giới hạn giá trị của d tối đa là 3.

<a><img src="https://imgur.com/QadpjUW.jpg" width="600" align="center"> 

In [27]:
with open('D:/Data Science/Data/Mutations/How Do We Locate Disease/Burrows and Wheeler Set Up Checkpoints/dataset_304_10 (4).txt') as f:
    lines = [line.rstrip() for line in f]
    f.close
text = lines[0] + '$'
patterns = lines[1:-1][0].split()
d = int(lines[-1])
partial_suffix_array, suffix_array_index = construct_partial_suffix_arrays(text, 1)
bwt = get_bwt(text)
print('The length of text: ', len(text))
print('Number of patterns: ', len(patterns))

The length of text:  10001
Number of patterns:  2000


In [28]:
import time
start_time = time.time()
starting_positions = InverseBurrowsWheelerTransform(bwt).get_approximate_pattern_matching_with_better_bwt(patterns, d, partial_suffix_array, suffix_array_index)
print("--- %s seconds ---" % (time.time() - start_time))

--- 50.273003816604614 seconds ---


In [135]:
import time
c = 100
start_time = time.time()
starting_positions = InverseBurrowsWheelerTransform(bwt).get_approximate_pattern_matching_with_check_points(patterns, c, d, partial_suffix_array, suffix_array_index)
print("--- %s seconds ---" % (time.time() - start_time))

--- 53.94083285331726 seconds ---


In [136]:
start_positions = []
for x in starting_positions:
    for el in x:
        start_positions.append(el)

print(sorted(start_positions))

[2, 3, 7, 9, 11, 13, 15, 21, 27, 31, 40, 40, 44, 50, 54, 54, 57, 61, 62, 69, 76, 95, 100, 102, 103, 106, 107, 119, 124, 129, 139, 145, 157, 186, 192, 192, 195, 197, 200, 212, 223, 223, 224, 229, 243, 244, 245, 261, 264, 270, 274, 287, 301, 304, 304, 306, 312, 313, 323, 327, 329, 330, 340, 346, 357, 359, 359, 359, 363, 364, 372, 379, 386, 388, 394, 403, 404, 409, 420, 428, 440, 441, 453, 458, 459, 460, 461, 462, 465, 473, 481, 485, 497, 509, 513, 513, 527, 530, 535, 536, 537, 539, 541, 547, 547, 553, 557, 562, 566, 568, 573, 582, 592, 593, 598, 601, 603, 607, 635, 640, 649, 652, 660, 661, 667, 671, 682, 696, 708, 709, 722, 726, 734, 740, 740, 746, 747, 764, 765, 772, 779, 781, 784, 789, 790, 803, 809, 821, 837, 839, 841, 844, 845, 849, 849, 850, 853, 853, 858, 867, 869, 882, 887, 902, 907, 910, 911, 912, 919, 919, 927, 929, 931, 935, 940, 941, 948, 953, 960, 960, 966, 980, 981, 988, 990, 991, 993, 994, 996, 996, 1002, 1005, 1008, 1021, 1022, 1023, 1023, 1024, 1032, 1033, 1050, 1056, 106

Bibliography: <br>
[15] Burrows, M., and D. J. Wheeler. “A Block-Sorting Lossless Data Compression Algorithm,” 1994.<br>
[16] Altschul, S. F., W. Gish, W. Miller, E. W. Myers, and D. J. Lipman. “Basic Local Alignment Search Tool.” <i>Journal of Molecular Biology</i> 215, no. 3 (October 5, 1990): 403–10. https://doi.org/10.1016/S0022-2836(05)80360-2.<br>
[17] An efficient implementation of the Burrows-Wheeler transform was described by Ferragina and Manzini, 2000<br>

In [15]:
class InverseBurrowsWheelerTransformNoneRecurrent(object):
    
    def __init__(self, bwt):
        self._bwt = bwt
        # Starting Positions of Patterns matching Text.
        self._start_positions = []
        # Generate first column from BWT.
        self._first_column = sorted(self._bwt)
        # Keeping last column in string type to speed up runtime.
        self._last_column_str = self._bwt
        # Last column in list type.
        self._last_column = list(self._bwt)
        # Get symbols of the first column.
        self._alphabet = np.unique(self._first_column)
        
        # Adding index to the first column.
        self._index_first_column = self.index_symbol_of_column(self._first_column)
        # Adding the same index of corresponding symbols to the last column.
        self._index_last_column = self.index_symbol_of_column(self._last_column)
        # Get index of the first symbol of the first column.
        self._first_occurence = self.get_first_occurence()
        
    # Inverting a bwt to original text
    def invert_bwt(self):
        
        org_string = []
        length = len(self._index_first_column)
        
        # Find row concluding symbol (0, '$') at last.
        index_str = self._index_last_column.index((0, '$'))
        
        while 1 < 2:
            # Finding the first symbol of the row above
            org_symbol_of_string = self._index_first_column[index_str]
            # Only choosing symbol, excluding index.
            org_string.append(org_symbol_of_string[1])
            # The loop is going on.
            index_str = self._index_last_column.index(org_symbol_of_string)
            # If the loop returns symbol (0, '$'), we stop the loop.
            if org_symbol_of_string == (0, '$'):
                break
                
        return(''.join(org_string))

    # Get approximatly patterns matching text by better bwt.
    def get_approximate_pattern_matching_with_better_bwt(self, patterns, d, partial_suffix_array, suffix_array_index):
        
        starting_positions = []
        
        for pattern in patterns:
            
            self._start_positions = []
            # Set initial top and bottom.
            top = 0
            l = len(self._last_column)
            bottom = l - 1
            
            """ If we want to assign the initial value of the pattern to match the symbol then run the hidden code folows:"""
            #symbol = pattern[-1]
            #pattern = pattern[0 : -1]

            #if symbol in self._last_column[top : (bottom + 1)]:

                #top_index = self._last_column[top : (bottom + 1)].index(symbol) + top
                #bottom_index = bottom - list(reversed(self._last_column[top : (bottom + 1)])).index(symbol)
                #top = self.get_last_to_first(top_index)
                #bottom = self.get_last_to_first(bottom_index)
                
            starting_positions.append(self.get_sub_aproximate_string(pattern, top, bottom, d, 0, 
                                                                     partial_suffix_array, suffix_array_index)
                                     )
        return(starting_positions)
    
    
    # Get approximatly patterns matching text by better bwt with checkpoints.
    def get_approximate_pattern_matching_with_check_points(self, patterns, c, d, partial_suffix_array, suffix_array_index):
        
        starting_positions = []
        self._index_check_point, self._count_dict = self.get_check_point_arrays(c)
        l = len(self._last_column)
      
        for pattern in patterns:
            self._start_positions = []
            # Set initial top and bottom.
            top = 0
            bottom = l - 1
            
            """ If we want to assign the initial value of the pattern to match the symbol then run the hidden code folows:"""
            #symbol = pattern[-1]
            #pattern = pattern[0 : -1]

            #if symbol in self._last_column[top : (bottom + 1)]:

                #top_index = self._last_column[top : (bottom + 1)].index(symbol) + top
                #bottom_index = bottom - list(reversed(self._last_column[top : (bottom + 1)])).index(symbol)
                #top = self.get_last_to_first(top_index)
                #bottom = self.get_last_to_first(bottom_index)
                
            starting_positions.append(self.get_sub_aproximate_string_with_check_points(pattern, top, bottom, d,
                                                                                       partial_suffix_array, suffix_array_index)
                                     )
        return(starting_positions)
    
    
    # Get approximatly pattern matching text by better bwt.
    def get_sub_aproximate_string(self, pattern, top, bottom, d, initial_mismatch, 
                                  partial_suffix_array, suffix_array_index
                                 ):
        # Get symbol exiting from top to bottom of the last column.
        apr_symbol_alphabet = ''.join(np.unique(self._last_column[top : (bottom + 1)]))
        # If pattern equals '', we find the positions of text that pattern matches the text.
        if pattern == '':
            
            for i in range(top, bottom + 1):
                # Set the initial backward count.
                count = 0
                last_index = i
                # When partial suffix array is not exist, we go backward again.
                while last_index not in suffix_array_index:
                    
                    count += 1
                    symbol = self._last_column_str[last_index]
                    last_index = self._first_occurence[symbol] + self.count_symbol(symbol, last_index)
            # Starting position of pattern equals sum of partial suffix array and extend backward count.        
            self._start_positions.append(partial_suffix_array[last_index] + count)
            
        else:
            symbol = pattern[-1] # Moving backward of Pattern.
            pattern = pattern[:-1] # Remove the last symbol of pattern.
            for apr_symbol in apr_symbol_alphabet:
                # Set the initial mismatch.
                mismatch = initial_mismatch
                
                if apr_symbol != symbol:
                    mismatch += 1
                # If mismatch <= d, we update top and bottom.
                if mismatch <= d:   
                    
                    new_top = top
                    new_bottom = bottom
                    
                    """ If we want to use normal bwt instead of better bwt then run the hidden code folows:"""
                    #top_index = self._last_column[new_top : (new_bottom + 1)].index(apr_symbol) + new_top
                    #bottom_index = new_bottom - list(reversed(self._last_column[new_top : (new_bottom + 1)])).index(apr_symbol)
                    #new_top = self.get_last_to_first(top_index)
                    #new_bottom = self.get_last_to_first(bottom_index)
                    
                    new_top = self._first_occurence[apr_symbol] + self.count_symbol(apr_symbol, new_top)
                    new_bottom = self._first_occurence[apr_symbol] + self.count_symbol(apr_symbol, new_bottom + 1) - 1
                    # Using recurence algorithm for the new patterns that are exact or in-exact matching text.
                    self.get_sub_aproximate_string(pattern, new_top, new_bottom, d, 
                                                                    mismatch, partial_suffix_array, suffix_array_index
                                                                    ) 
        return(self._start_positions)
    
    # Get approximatly pattern matching text by better bwt with checkpoints.
    def get_sub_aproximate_string_with_check_points(self, pattern, top, bottom, d,
                                                    partial_suffix_array, suffix_array_index
                                                    ):
        # Get symbol exiting from top to bottom of the last column.
        # apr_symbol_alphabet = ''.join(np.unique(self._last_column[top : (bottom + 1)])) # f(x)
        # If pattern equals '', we find the positions of text that pattern matches the text.
        #self._index_check_point, self._count_dict = self.get_check_point_arrays(1)
        # Set the initial mismatch.
       
        top_bottom_update = [[top, bottom, 0, 0], [1, 1, 1, -1]]
        count = 0
        while top_bottom_update != [[1, 1, 1, -1]]:
            
            t_bt = top_bottom_update[0]
            
            top = t_bt[0]
            bottom = t_bt[1]
            initial_mismatch = t_bt[2]
            label = t_bt[3]
            new_label = top_bottom_update[1][3]
  
            #print(top, bottom, initial_mismatch)
            apr_symbol_alphabet = ''.join(np.unique(self._last_column[top : (bottom + 1)]))
            
            if pattern != '': # E(x)
                top_bottom_update.pop(0)
                symbol = pattern[-1]  # A(x)
                
                if label != new_label:
                    pattern = pattern[:-1] # f(x)
                
                for apr_symbol in apr_symbol_alphabet:
                    count = label + 1
                    mismatch = initial_mismatch
                    new_top = top
                    new_bottom = bottom

                    if apr_symbol != symbol:
                        mismatch += 1 # f(x)
                    #print(apr_symbol, symbol, mismatch)
                    # If mismatch <= d, we update top and bottom.
                    if mismatch <= d:   
                        # Updating top and bottom using checkpoints.
                        if new_top not in self._index_check_point:
                            new_top = self._first_occurence[apr_symbol] + self.count_symbol(apr_symbol, new_top)
                        else:
                            new_top = self._first_occurence[apr_symbol] + self._count_dict[new_top][apr_symbol]

                        if new_bottom + 1 not in self._index_check_point:
                            new_bottom = self._first_occurence[apr_symbol] + self.count_symbol(apr_symbol, new_bottom + 1) - 1 
                        else:
                            new_bottom = self._first_occurence[apr_symbol] + self._count_dict[new_bottom + 1][apr_symbol] - 1
                    
                        top_bottom_update.insert(-1, [new_top, new_bottom, mismatch, count])
                        #print('new_top_bottom', new_top, new_bottom)
                        #print(top_bottom_update)
                                   
            else:
                new_top_bottom_update = top_bottom_update[0 : -1]
                for x in new_top_bottom_update:
                    
                    top = x[0]
                    bottom = x[1]
                    for i in range(top, bottom + 1): # D(x)
                        # Set the initial backward count.
                        count = 0
                        last_index = i

                        while last_index not in suffix_array_index:

                            count += 1
                            symbol = self._last_column_str[last_index]
                            last_index = self._first_occurence[symbol] + self.count_symbol(symbol, last_index)

                        self._start_positions.append(partial_suffix_array[last_index] + count)
                
                return (self._start_positions)    
        return('None')
    
    # Adding index for each symbol.
    def index_symbol_of_column(self, column):
       
        l = len(column)
        col = column[::]
        
        for x in self._alphabet:
            count = 0
            for i in range(l):
                # Index equals occurence order of symbol.
                if col[i] == x:
                    count = count + 1
                    col[i] = (count, x)
                    
        return(col) 
    # Finding index of symbol in the first column coresponds to the same symbol in the last column.
    def get_last_to_first(self, last_index):
        
        first_index = self._index_first_column.index(self._index_last_column[last_index])
        
        return (first_index)
    # Finding index of the symbol occuring at first in the first column.
    def get_first_occurence(self):
        first_occurence = {}
        for x in self._alphabet:
            first_occurence.update({x : self._first_column.index(x)})
        return(first_occurence)
    # Count symbol from 0 to index.
    def count_symbol(self, symbol, index):
        
        return(self._last_column_str.count(symbol, 0, index))
    # Construct checkpoints arrays.
    def get_check_point_arrays(self, c):
        
        length = len(self._last_column)
        index_check_point = np.arange(0, length + 1, c).tolist()
        count_dict = {}

        for i in index_check_point: 
            check_point_arrays = {}
            instance_column = self._last_column[0: i]
            # Finding number of symbols' occurence from 0 to i.
            count_dict.update({i:Counter(instance_column)})
        
        return(index_check_point, count_dict)   

    def divide_pattern(self, pattern, d):
        sub_strings = []
        k = int(len(pattern)/(d+1))
        for i in range(0, d):
            sub_strings.append(pattern[i*k: ((i+1)*k)])
        sub_strings.append(pattern[d*k::])
        return(sub_strings)

In [16]:
text = 'ACATGCTACTTT$'
patterns = ['ATT', 'GCC', 'GCTA', 'TATT']
pattern = patterns[0]
d = 1
partial_suffix_array, suffix_array_index = construct_partial_suffix_arrays(text, 1)
bwt = get_bwt(text)
top = 0
bottom = len(text) - 1
import time
c = 1
start_time = time.time()
starting_positions = InverseBurrowsWheelerTransformNoneRecurrent(bwt).get_approximate_pattern_matching_with_check_points(patterns, c, d, partial_suffix_array, suffix_array_index)
#starting_positions = InverseBurrowsWheelerTransformNoneRecurrent(bwt).get_sub_aproximate_string_with_check_points(pattern, top, bottom, d, 
                                                    #partial_suffix_array, suffix_array_index
                                                    #)
print("--- %s seconds ---" % (time.time() - start_time))
starting_positions

--- 0.0029985904693603516 seconds ---


[[2, 7, 8, 9], [4], [4], [6]]

In [145]:
bwt

'T$TCAGATTCATC'

In [17]:
with open('D:/Data Science/Data/Mutations/How Do We Locate Disease/Burrows and Wheeler Set Up Checkpoints/dataset_304_10 (4).txt') as f:
    lines = [line.rstrip() for line in f]
    f.close
text = lines[0] + '$'
patterns = lines[1:-1][0].split()
d = int(lines[-1])
partial_suffix_array, suffix_array_index = construct_partial_suffix_arrays(text, 1)
bwt = get_bwt(text)
print('The length of text: ', len(text))
print('Number of patterns: ', len(patterns))

The length of text:  10001
Number of patterns:  2000


In [18]:
import time
c = 100
start_time = time.time()
starting_positions = InverseBurrowsWheelerTransformNoneRecurrent(bwt).get_approximate_pattern_matching_with_check_points(patterns, c, d, partial_suffix_array, suffix_array_index)
print("--- %s seconds ---" % (time.time() - start_time))

--- 53.70004391670227 seconds ---


In [19]:
start_positions = []
for x in starting_positions:
    for el in x:
        start_positions.append(el)

print(sorted(start_positions))

[2, 3, 7, 9, 11, 13, 15, 21, 27, 31, 40, 40, 44, 50, 54, 54, 57, 61, 62, 69, 76, 95, 100, 102, 103, 106, 107, 119, 124, 129, 139, 145, 157, 186, 192, 192, 195, 197, 200, 212, 223, 223, 224, 229, 243, 244, 245, 261, 264, 270, 274, 287, 301, 304, 304, 306, 312, 313, 323, 327, 329, 330, 340, 346, 357, 359, 359, 359, 363, 364, 372, 379, 386, 388, 394, 403, 404, 409, 420, 428, 440, 441, 453, 458, 459, 460, 461, 462, 465, 473, 481, 485, 497, 509, 513, 513, 527, 530, 535, 536, 537, 539, 541, 547, 547, 553, 557, 562, 566, 568, 573, 582, 592, 593, 598, 601, 603, 607, 635, 640, 649, 652, 660, 661, 667, 671, 682, 696, 708, 709, 722, 726, 734, 740, 740, 746, 747, 764, 765, 772, 779, 781, 784, 789, 790, 803, 809, 821, 837, 839, 841, 844, 845, 849, 849, 850, 853, 853, 858, 867, 869, 882, 887, 902, 907, 910, 911, 912, 919, 919, 927, 929, 931, 935, 940, 941, 948, 953, 960, 960, 966, 980, 981, 988, 990, 991, 993, 994, 996, 996, 1002, 1005, 1008, 1021, 1022, 1023, 1023, 1024, 1032, 1033, 1050, 1056, 106

### 3 Hidden Markov Models<a name = 'hmm'></a>

#### 3.3 Viterbi Algorithm

In [137]:
import numpy as np
import pandas as pd

In [396]:
trans = pd.read_csv('D:/Data Science/Data/Mutations/How Do We Locate Disease/Viterbi/trans.txt', sep = '\s+')
trans

,A,B,C,D
A,0.144,0.219,0.383,0.254
B,0.232,0.217,0.062,0.489
C,0.107,0.290,0.302,0.301
D,0.099,0.157,0.258,0.486


In [397]:
emis = pd.read_csv('D:/Data Science/Data/Mutations/How Do We Locate Disease/Viterbi/emis.txt', sep = '\s+')
emis

,x,y,z
A,0.600,0.126,0.274
B,0.059,0.591,0.350
C,0.184,0.575,0.241
D,0.442,0.440,0.118


In [398]:
Sigma = 'yyxxyxxxxyxyxxzyzyyxxyyxzyxyyxzzzzzxzzzzzyzxxxzzxzzxyyxyyxzyxxyzxzzzxzxyzzxxzyzyyyzxxxxyzxyxyyzxzxxx'
states = 'ABCD'

In [399]:
transition = {}
emission = {}

for x in trans.index:
    tr = {}
    for y in trans.columns:
        tr.update({y : trans.loc[x, y]})
    transition.update({x : tr})    
    
for x in emis.index:
    em = {}
    for y in emis.columns:
        em.update({y : emis.loc[x, y]})
    emission.update({x : em}) 

In [400]:
def get_viterbi_algorithm(Sigma, states, transition, emission):
    
    n =  len(Sigma)
    source_prop = 1/len(states)
    back_track = {}
    path = []
    s = {}
    
    # Finding s_{k, source}
    for k in states:
        s.update({k : {0 : source_prop*emission[k][Sigma[0]]}})
    # Finding s_{k, i}
    for i in range(1, n):
        sub_back_track = {}
        
        for k in states:    
            prop = []
            _emis = emission[k][Sigma[i]]
            
            for l in states:
                prop.append(s[l][i-1]*transition[l][k]*_emis)
               
            s[k].update({i : max(prop)})
            
            sub_back_track.update({k : states[np.argmax(prop)]})
        # Get back_tracking 
        back_track.update({i : sub_back_track})
    
    prop = []    
    
    # Get the last state.
    for l in states:
        prop.append(s[l][n-1])
    state = states[np.argmax(prop)]
    
    # Find the path of states.
    path.append(state)

    for i in reversed(range(1, n)):
        state = back_track[i][state] 
        path.append(state)
        
    return(''.join(reversed(path)))    

In [401]:
get_viterbi_algorithm(Sigma, states, transition, emission)

'BDDDDDDDDDDDDDCCBDDDDDDDDDDDDDCBACBDCBBBACBDDDCBDCBDDDDDDDDDDDCBDCBBDDDDCBDDCCBDDCBDDDDCBDDDDCBDDDDD'